In [29]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [30]:
pd.set_option('display.max_columns', None)

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as RFC
# from xgboost import XGBClassifier as XGB
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import ParameterGrid
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 1. Train 이상치 찾아보기
- 밀려적힘
- 이상한 값이 적힘
- 객관식 보기가 아닌 다른숫자
- 중복대답


In [59]:
meta2017 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_Original/KNOW_2017.csv')
meta2018 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_Original/KNOW_2018.csv')
meta2019 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_Original/KNOW_2019.csv')
meta2020 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_Original/KNOW_2020.csv')

## 1-1. 학습에서 필요없는 knowcode 제거

In [33]:
#knowcode 결측치 제거
meta2017 = meta2017[meta2017['knowcode'] != 9999999]
meta2018 = meta2018[meta2018['knowcode'] != 9999999]
meta2019 = meta2019[meta2019['knowcode'] != 9999999]
meta2020 = meta2020[meta2020['knowcode'] != 9999999]

## 1-2. 이상치 및 결측치 찾기


컬럼별로 돌아가며 이상한 애들을 찾는다

In [34]:
for col in meta2017.columns[129:140]: #여기서 연도별로 슬라이스 해줘가며 컬럼별로 유니크값을 본다
# for col in meta2018.columns[129:140]:
# for col in meta2019.columns[129:140]:
# for col in meta2020.columns[129:140]:
  print(col)
  L = meta2017[col].unique()
  # L = meta2018[col].unique()
  # L = meta2019[col].unique()
  # L = meta2020[col].unique()
  try:
    L.sort() #값들이 순서대로 나옴 -> 가독성 좋게보려고
    if len(L) < 10:
      print(L)
    else:
      for i in range(10,len(L),10):
        print(L[i-10:i])
      print(L[i:])
      print()
    print()
  except:
    print(f'{col} 이상함 발견됨!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!') #이건 sort안되는걸 방지
    print()

# EX) 예를 들어 보기가 1~5 번부터 인데 여기서 나온 값이 [1,2,3,4,5,8, 2900] 이면 이상한 값이 껴있다고 유추가능

bq24_4
[1 2]

bq24_5
[1 2]

bq24_6
[1 2]

bq24_7
[1 2]

bq24_8
[1 2]

bq25
[1 2 3 4 5]

bq26
[1 2 3 4 5]

bq27
[1 2 3 4 5]

bq28
[1 2 3 4 5]

bq29
[1 2 3 4 5]

bq30
['(수학)선생님,(수학)교사' '(영업)사원, (판매)사원' '(영업/판매)사원' '(의사)선생님' '(체육)선생님,교사'
 '(판매)사원' '00바텐더님' '00반 선생님' '00씨, 00기사' '00씨,사장님']
['119' '119구급대원' '3D' '3D프로그래머,3D아티스트,컴퓨터브로그래머TD,그랙픽아티스트' 'A/S 기사' 'A/S기사'
 'AE' 'CG(컴퓨터 그래픽)' 'CPA' 'CPA, 공인회계사, 회계사']
['CTO' 'DJ / 아나듀오 / 아나듀서 / MC' 'Design engineer' 'Dr, 의사' 'FC' 'FC, 설계사'
 'FD' 'Fab 총괄 관리자, 장비실장, 안전관리자' 'IT서비스 헬퍼' 'MC, 사회자']
['MC,마술사,마술강사,일루셔니스트' 'NDT, NDE' 'NGO단체활동가, 단체의 감사' 'OP' 'PB'
 'PB, 자산관리사, 증권브로커, 금융상품상담사' 'PCO' 'PC엔지니어' 'PD' 'PD, 감독']
['PD, 제작가' 'PD, 프로듀서' 'PD, 프로듀서, 연출가' 'PD,감독님' 'PM' 'PR인' 'RF엔지니어' 'S.H'
 'SOC engineer, ASIC enginee, Front-End Engineer, Inplementation Engineer'
 'VTS']
['ca' 'chef' 'chef,요리사' 'color cordinator' 'operater(운전원)' '~씨' '가공기사'
 '가방디자이너' '가수' '가스안전관리사']
['가스안전관리자' '가이드' '가이드, 선생님' '가이드,선생님' '가이드님' '가이드님, 선생님' '가정부' '간병사' '간병인'
 '간판공']
['간호사' '간

## 1-3. Column 순회하면서 발견한점 정리
- 객관식답에 주관식 답이 들어가 있거나
- 주관식 답에 객관식 답이 들어가 있는걸 심심치 않게 볼 수 있다
- 이에따라 몇몇 Row 값들이 어느 부분에서 밀려 적힌곳이 있을 거라고 유추가능
- 밀려적힌 값들을 우선 찾는게 중요하다고 판단됨


성별을 묻는 질문 보기 1,2만 올 수 있다

In [35]:
meta2018['bq35'] #

0       1
1       1
2       1
3       1
4       2
       ..
9067    1
9068    1
9069    2
9070    1
9071    1
Name: bq35, Length: 9072, dtype: object

그러나 이렇게 1,2 말고 다른 값들이 있다

bq36이 나이를 주관식으로 입력하는 질문인데 거기서 밀려 적힌듯 하다

In [36]:
meta2018['bq35'].unique() 

array([1.0, 2.0, 27.0, 40.0, 31.0, 53.0, 39.0, 26.0, 32.0, 52.0, 50.0,
       42.0, 35.0, 46.0, 37.0, 25.0, 44.0, 47.0, 56.0, '1', '2',
       ' 라이트룸 등의 컴퓨터 프로그램"', '42', '33', '45', '38', '30'], dtype=object)

위에 코드에 따라 bq 36 질문이 bq35에다가 한칸 앞으로 밀려 적힌것을 알 수 있다

In [37]:
meta2018[meta2018['bq35'] == 27.0]

,idx,cq1,cq2,cq3,cq4,cq5,cq6,cq7,cq8,cq9,cq10,cq11,cq12,cq13,cq14,cq15,cq16,cq17,cq18,cq19,cq20,cq21,cq22,cq23,cq24,cq25,cq26,cq27,cq28,cq29,cq30,cq31,cq32,cq33,cq34,cq35,cq36,cq37,cq38,cq39,cq40,cq41,cq42,cq43,cq44,cq45,cq46,cq47,cq48,cq49,cq50_1,cq50_2,cq50_3,cq50_4,cq50_5,cq50_6,cq50_7,cq50_8,iq1,iq2,iq3,iq4,iq5,iq6,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13,bq14,bq15,bq16,bq17,bq18,bq19,bq20,bq21,bq221,bq222,bq223,bq231,bq232,bq233,bq234,bq235,bq241,bq242,bq243,bq244,bq245,bq25,bq25_1,bq26_1,bq26_1a,bq26_2,bq26_2a,bq26_3,bq26_3a,bq26_4,bq26_4a,bq27,bq28,bq28_1,bq29,bq30,bq31,bq32,bq33,bq34,bq35,bq36,bq37,bq37_1,bq38,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode
456,9942,3,5,5,5,2,5,2,4,3,3,1,1,5,5,5,4,4,4.0,4.0,5.0,4.0,4.0,4.0,2.0,5.0,2.0,5.0,3.0,3.0,2.0,4.0,2.0,2.0,5.0,5.0,5.0,3.0,1.0,2.0,4.0,3.0,2.0,1.0,2.0,3.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,3.0,1.0,4.0,2.0,5.0,4.0,2.0,6.0,1.0,전기기사,,,1,6,전기설비유지보수,7,2.0,4.0,4.0,4.0,4.0,3.0,4.0,2.0,3,5,5,4,1.0,1.0,1.0,2.0,3.0,1.0,1.0,4.0,80.0,1,,0,1,0,0.0,0.0,,1,,,,,1,2,1,2,1,2,1,2,1,1,2,2.0,자동화,,Auto CAD,,없음,,3,1,27,4,전자전기학,1,1,,1,40,3000.0,2700,,NaN,831101
6512,16005,2,5,5,5,5,5,5,5,5,5,4,3,5,5,1,3,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,2.0,2.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,1.0,5.0,5.0,3.0,1.0,3.0,5.0,1.0,1.0,4.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,3.0,3.0,4.0,3.0,3.0,1.0,3.0,2.0,,,,1,2,프로그램 툴사용.제품구조 이해,5,4.0,4.0,4.0,3.0,4.0,4.0,2.0,4.0,4,4,4,3,2.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,100.0,1,,0,1,0,0.0,0.0,,,2,,,,1,2,1,2,1,2,1,2,1,1,2,2.0,기술의 발전으로 적은 인원으로 일처리가 가능,,프로그램,,제품설계,3D프린터,3,1,27,4,기계공학,1,1,,1,52,2900,2300,,NaN,821101


< 다른 발견한 점 >
1. 이렇게 밀려 적힌값을 보니 맨뒤가 Nan으로 입력되어 있다.
2. 이 설문지는 결측값이 있으면 ' '로 저장하는데, 밀려 적힌 값이 있는 Row는 저렇게 맨뒤가 Nan이 존재한다
3. Nan이 있는 Row들을 조사하면 밀려적힌 값들을 찾을 수 있을 듯 하다
4. Nan 값이 있는 Column들을 찾아 Nan이 있는 Row들을 추적한다

## 1-4. 연도별 조사

2017년 : 딱히 문제되는걸 못찾음

In [38]:
c_list = meta2017.columns.tolist()
nan2017 = []

for i in c_list:
    nan2017.append([i,list(meta2017[meta2017[i].isnull()].index)])
nan2017 #2017년에서는 발견되지 않음

[['idx', []],
 ['aq1_1', []],
 ['aq1_2', []],
 ['aq2_1', []],
 ['aq2_2', []],
 ['aq3_1', []],
 ['aq3_2', []],
 ['aq4_1', []],
 ['aq4_2', []],
 ['aq5_1', []],
 ['aq5_2', []],
 ['aq6_1', []],
 ['aq6_2', []],
 ['aq7_1', []],
 ['aq7_2', []],
 ['aq8_1', []],
 ['aq8_2', []],
 ['aq9_1', []],
 ['aq9_2', []],
 ['aq10_1', []],
 ['aq10_2', []],
 ['aq11_1', []],
 ['aq11_2', []],
 ['aq12_1', []],
 ['aq12_2', []],
 ['aq13_1', []],
 ['aq13_2', []],
 ['aq14_1', []],
 ['aq14_2', []],
 ['aq15_1', []],
 ['aq15_2', []],
 ['aq16_1', []],
 ['aq16_2', []],
 ['aq17_1', []],
 ['aq17_2', []],
 ['aq18_1', []],
 ['aq18_2', []],
 ['aq19_1', []],
 ['aq19_2', []],
 ['aq20_1', []],
 ['aq20_2', []],
 ['aq21_1', []],
 ['aq21_2', []],
 ['aq22_1', []],
 ['aq22_2', []],
 ['aq23_1', []],
 ['aq23_2', []],
 ['aq24_1', []],
 ['aq24_2', []],
 ['aq25_1', []],
 ['aq25_2', []],
 ['aq26_1', []],
 ['aq26_2', []],
 ['aq27_1', []],
 ['aq27_2', []],
 ['aq28_1', []],
 ['aq28_2', []],
 ['aq29_1', []],
 ['aq29_2', []],
 ['aq30_1', []],
 

2018년 조사 : nan 값 있는 row 발견

In [39]:
c_list = meta2018.columns.tolist()
nan2018 = []

for i in c_list:
    nan2018.append([i,list(meta2018[meta2018[i].isnull()].index)])
nan2018

[['idx', []],
 ['cq1', []],
 ['cq2', []],
 ['cq3', []],
 ['cq4', []],
 ['cq5', []],
 ['cq6', []],
 ['cq7', []],
 ['cq8', []],
 ['cq9', []],
 ['cq10', []],
 ['cq11', []],
 ['cq12', []],
 ['cq13', []],
 ['cq14', []],
 ['cq15', []],
 ['cq16', []],
 ['cq17', []],
 ['cq18', []],
 ['cq19', []],
 ['cq20', []],
 ['cq21', []],
 ['cq22', []],
 ['cq23', []],
 ['cq24', []],
 ['cq25', []],
 ['cq26', []],
 ['cq27', []],
 ['cq28', []],
 ['cq29', []],
 ['cq30', []],
 ['cq31', []],
 ['cq32', []],
 ['cq33', []],
 ['cq34', []],
 ['cq35', []],
 ['cq36', []],
 ['cq37', []],
 ['cq38', []],
 ['cq39', []],
 ['cq40', []],
 ['cq41', []],
 ['cq42', []],
 ['cq43', []],
 ['cq44', []],
 ['cq45', []],
 ['cq46', []],
 ['cq47', []],
 ['cq48', []],
 ['cq49', []],
 ['cq50_1', []],
 ['cq50_2', []],
 ['cq50_3', []],
 ['cq50_4', []],
 ['cq50_5', []],
 ['cq50_6', []],
 ['cq50_7', []],
 ['cq50_8', []],
 ['iq1', []],
 ['iq2', []],
 ['iq3', []],
 ['iq4', []],
 ['iq5', []],
 ['iq6', []],
 ['bq1', []],
 ['bq2', []],
 ['bq3', []]

각 직업군별로 값이 입력된다고 가정하고
Knowcode 별로 Nan이 있는 Row들을 모아봤다.

In [40]:
meta2018.loc[[406, 1253,1548,1628,1946,2938,3064,
              3505,4000,4076,4254,4478,4718,4884,
              5292,5433,5574,5784,5902,6094,6465,
              6512,6685,7480,7815,7920,7992,8171,
              8205,8760,8783,8987,8992,9016]]['knowcode'].value_counts()

821101    16
831101    14
562101     1
415404     1
622304     1
131201     1
Name: knowcode, dtype: int64

공교롭게도 특정 직업군에 Nan값이 몰려있는 경우가 많았다

In [41]:
meta2018[meta2018.knowcode == 821101]

,idx,cq1,cq2,cq3,cq4,cq5,cq6,cq7,cq8,cq9,cq10,cq11,cq12,cq13,cq14,cq15,cq16,cq17,cq18,cq19,cq20,cq21,cq22,cq23,cq24,cq25,cq26,cq27,cq28,cq29,cq30,cq31,cq32,cq33,cq34,cq35,cq36,cq37,cq38,cq39,cq40,cq41,cq42,cq43,cq44,cq45,cq46,cq47,cq48,cq49,cq50_1,cq50_2,cq50_3,cq50_4,cq50_5,cq50_6,cq50_7,cq50_8,iq1,iq2,iq3,iq4,iq5,iq6,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13,bq14,bq15,bq16,bq17,bq18,bq19,bq20,bq21,bq221,bq222,bq223,bq231,bq232,bq233,bq234,bq235,bq241,bq242,bq243,bq244,bq245,bq25,bq25_1,bq26_1,bq26_1a,bq26_2,bq26_2a,bq26_3,bq26_3a,bq26_4,bq26_4a,bq27,bq28,bq28_1,bq29,bq30,bq31,bq32,bq33,bq34,bq35,bq36,bq37,bq37_1,bq38,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode
1253,10741,4,5,4,5,4,4,3,4,3,3,3,2,5,5,5,3,4,5.0,4.0,5.0,4.0,4.0,2.0,1.0,1.0,1.0,3.0,3.0,3.0,2.0,3.0,1.0,3.0,5.0,3.0,3.0,2.0,1.0,3.0,3.0,3.0,3.0,1.0,2.0,3.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,3.0,3.0,3.0,3.0,1.0,3.0,2.0,,,,1,2,직무관련 실습,3,2.0,3.0,3.0,3.0,4.0,4.0,4.0,3.0,3,3,3,3,1.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,100.0,1,,0,1,0,0.0,0.0,,1,,,,,1,2,1,2,1,2,1,2,1,1,2,2.0,기계의 자동화로 감소,없다,청진기,없다,없다,없다,4,2,40,2,정보처리,1,1,,1,45,3000.0,2000,,NaN,821101
1548,11036,1,1,1,1,3,3,1,2,1,1,2,1,5,2,3,4,5,5.0,1.0,5.0,4.0,4.0,1.0,1.0,5.0,5.0,5.0,2.0,3.0,3.0,3.0,2.0,5.0,5.0,4.0,3.0,3.0,1.0,3.0,3.0,4.0,1.0,1.0,3.0,3.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,2.0,3.0,3.0,3.0,1.0,3.0,1.0,금속재료시험 기능사,,,2,,,7,2.0,3.0,2.0,1.0,3.0,3.0,3.0,2.0,2,3,3,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,100.0,1,,0,1,0,0.0,0.0,,,2,,,,1,2,1,2,1,2,1,2,1,1,2,3.0,철강제조업의 불황으로 큰변화는 없을것 같아서,,없음,,,,2,1,31,2,금속,1,3,,1,48,2800.0,2500,,NaN,821101
1628,11116,3,1,1,1,2,4,1,2,2,2,2,2,1,5,1,4,3,3.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,4.0,4.0,5.0,5.0,2.0,2.0,2.0,1.0,3.0,4.0,4.0,1.0,3.0,3.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,4.0,2.0,4.0,4.0,4.0,3.0,1.0,2.0,2.0,,,,2,,,5,2.0,3.0,2.0,2.0,4.0,2.0,2.0,3.0,3,3,3,3,1.0,1.0,1.0,2.0,1.0,1.0,2.0,4.0,100.0,1,,0,1,0,0.0,0.0,,1,,,,,1,1,1,2,1,2,1,2,1,1,2,2.0,1차산업이 늘어나지는 않을것이다,,없다,,,,3,1,53,2,이,1,1,,1,60,5000.0,4000,,NaN,821101
3505,12996,2,2,1,1,2,2,2,2,2,3,2,2,4,2,1,4,4,3.0,3.0,3.0,2.0,4.0,2.0,1.0,4.0,3.0,2.0,2.0,4.0,2.0,3.0,2.0,5.0,4.0,4.0,3.0,3.0,2.0,3.0,4.0,4.0,3.0,3.0,3.0,2.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,3.0,1.0,2.0,2.0,2.0,3.0,1.0,5.0,2.0,,,,2,,,3,1.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,1,3,4,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,100.0,1,,0,1,0,0.0,0.0,,1,,,,,1,1,1,2,1,2,1,2,1,2,2,3.0,3D업종이라 할려는 사람들이 줄어들고 있어서,,특별히 없다,판금원,,,1,1,52,2,공고졸업,1,1,,1,48,4900.0,2400,,NaN,821101
4000,13491,4,5,4,5,5,4,4,3,5,4,3,2,5,3,5,3,5,3.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,3.0,1.0,1.0,1.0,5.0,1.0,4.0,2.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,4.0,1.0,1.0,1.0,4.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,1.0,5.0,1.0,제강기능사,지게차,,1,3,지게차 자격증,2,2.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,3,3,3,3,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,90.0,1,,0,1,0,0.0,0.0,,1,,,,,1,2,1,2,1,2,1,2,1,1,2,4.0,아파트 및 교량 건설이 많을것이므로,용해조업자,잘모름,없음,현장관리감독자,없음,5,1,50,2,기계,1,1,,1,52,8000.0,5000,,NaN,821101
4076,13568,4,5,4,5,5,4,4,3,5,4,3,2,5,3,5,3,5,3.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,2.0,1.0,1.0,1.0,5.0,1.0,4.0,2.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,4.0,1.0,1.0,1.0,4.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,1.0,5.0,1.0,제강기능사,지게차자격증,,1,3,지게차자격증,2,2.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,3,3,3,3,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,90.0,1,,0,1,0,0.0,0.0,,1,,,,,1,2,1,2,1,2,1,2,1,1,2,4.0,아파트 및 교량 건설이 많을 것이므로,용해 조업자,없음,회사원(비디오 모니터),현장 관리 감독자,없음,5,1,50,2,이과,1,1,,1,52,8000.0,5000,,NaN,821101
4718,14210,1,1,1,1,3,3,2,2,1,1,1,1,5,1,1,4,5,5.0,1.0,5.0,3.0,2.0,1.0,1.0,5.0,5.0,5.0,1.0,4.0,3.0,3.0,3.0,5.0,5.0,4.0,3.0,3.0,2.0,3.0,4.0,4.0,2.0,1.0,3.0,3.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,4.0,3.0,3.0,3.0,3.0,1.0,3.0,2.0,,,,2,,,2,3.0,3.0,3.0,3.0,3.0,3.0,2.0,3.0,1,3,3,3,1.0,1.0,1.0,2.0,1.0,1.0,1.0,5.0,100.0,1,,0,1,0,0.0,0.0,,1,,,,,1,2,1,1,1,1,1,1,1,1,2,3.0,일이 힘이 들어서 할려는 사람이 없다,,제어시스템,

1. 직업코드 821101의 경우 값이 밀려있는게 확실하나 어디부터 밀린건지는 알수가 없다
2. 설문지의 정보와 비교하여 밀린값을 유추해본다
- cp46보기 : 1,2,3,4,5
- cp47보기 : 1,2,3
- cp48보기 : 1,2
- cp49보기 : 1,2,3,4,5
- cp50_1보기 : 1,2,3

3. 현재 데이터 상황
- cp46 : 1 만있음
- cp47 : 1 만있음
- cp48 : 2보다 큰값이 있음
- cp49 : 1,3이 있음
- cp50_1 : 4보다 작은 값들임

4. 이에따라 cp49의 답변이 cp48로 밀려적혀 있음을 확실하게 알 수 있다

5. 고로 cp48을 결측으로 놔두고 모두 한칸씩 뒤로 민다

6. 이전부터 밀렸을수도 있으나 일단 확실한 지점부터 밀린값들을 옮기는 작업을 하는게 맞는다 생각했고, cp48 이전 질문이 밀려적혔더라도 같은 직업간의 개별적인 특징은 고대로 담고있는거라 집요하게 밀린시점을 찾는것을 여기서 마치자 했다

831101 도 821101 같은 양상으로 밀려있어 위와같이 밀린값을 처리하는 작업을 한다

In [42]:
meta2018[meta2018.knowcode == 831101]

,idx,cq1,cq2,cq3,cq4,cq5,cq6,cq7,cq8,cq9,cq10,cq11,cq12,cq13,cq14,cq15,cq16,cq17,cq18,cq19,cq20,cq21,cq22,cq23,cq24,cq25,cq26,cq27,cq28,cq29,cq30,cq31,cq32,cq33,cq34,cq35,cq36,cq37,cq38,cq39,cq40,cq41,cq42,cq43,cq44,cq45,cq46,cq47,cq48,cq49,cq50_1,cq50_2,cq50_3,cq50_4,cq50_5,cq50_6,cq50_7,cq50_8,iq1,iq2,iq3,iq4,iq5,iq6,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13,bq14,bq15,bq16,bq17,bq18,bq19,bq20,bq21,bq221,bq222,bq223,bq231,bq232,bq233,bq234,bq235,bq241,bq242,bq243,bq244,bq245,bq25,bq25_1,bq26_1,bq26_1a,bq26_2,bq26_2a,bq26_3,bq26_3a,bq26_4,bq26_4a,bq27,bq28,bq28_1,bq29,bq30,bq31,bq32,bq33,bq34,bq35,bq36,bq37,bq37_1,bq38,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode
456,9942,3,5,5,5,2,5,2,4,3,3,1,1,5,5,5,4,4,4.0,4.0,5.0,4.0,4.0,4.0,2.0,5.0,2.0,5.0,3.0,3.0,2.0,4.0,2.0,2.0,5.0,5.0,5.0,3.0,1.0,2.0,4.0,3.0,2.0,1.0,2.0,3.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,3.0,1.0,4.0,2.0,5.0,4.0,2.0,6.0,1.0,전기기사,,,1,6,전기설비유지보수,7,2.0,4.0,4.0,4.0,4.0,3.0,4.0,2.0,3,5,5,4,1.0,1.0,1.0,2.0,3.0,1.0,1.0,4.0,80.0,1,,0,1,0,0.0,0.0,,1,,,,,1,2,1,2,1,2,1,2,1,1,2,2.0,자동화,,Auto CAD,,없음,,3,1,27,4,전자전기학,1,1,,1,40,3000.0,2700,,NaN,831101
1946,11434,2,5,4,4,5,4,4,3,3,2,3,2,5,4,1,3,4,4.0,1.0,4.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,5.0,1.0,1.0,1.0,1.0,2.0,4.0,3.0,4.0,3.0,2.0,3.0,4.0,4.0,3.0,3.0,1.0,3.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,4.0,3.0,3.0,3.0,4.0,2.0,6.0,1.0,전기철도기사,전기기사,전기공사기사,1,2,전기철도개요및 시공실습 등,4,5.0,4.0,3.0,3.0,4.0,4.0,4.0,4.0,3,4,5,4,3.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,70.0,1,,0,1,0,0.0,0.0,,1,,,,,1,2,1,2,1,2,1,2,1,1,2,2.0,4차 산업혁명및 자동화 등,,"캐드,엑셀 등",고천정 전등 설치업체,전기철도공사 감리 및 설계업체,,4,1,39,4,전기공학,1,1,,1,52,5200.0,2500,,NaN,831101
2938,12429,1,5,4,3,3,4,4,4,4,3,3,3,5,3,1,3,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,5.0,1.0,1.0,3.0,1.0,2.0,3.0,2.0,4.0,2.0,1.0,1.0,4.0,4.0,3.0,2.0,3.0,3.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,2.0,2.0,4.0,2.0,6.0,2.0,,,,1,2,"직무교육, 현장안전교육, 도구사용법",4,5.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,3,3,3,3,2.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,70.0,1,,0,1,0,0.0,0.0,,1,,,,,1,2,3,,3,,3,,3,,2,3.0,"전산화,인공지능도 있지만 수작업필요",,CAD ian,,,,3,1,26,4,전자공학,1,1,,1,40,3500.0,3450,,NaN,831101
4254,13746,2,5,5,5,4,4,3,4,4,4,3,4,5,4,2,3,2,3.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,4.0,2.0,2.0,4.0,2.0,4.0,2.0,4.0,3.0,3.0,3.0,4.0,2.0,3.0,3.0,3.0,3.0,2.0,2.0,4.0,2.0,1.0,3.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,3.0,3.0,2.0,3.0,3.0,4.0,8.0,2.0,6.0,1.0,전기기사,,,2,,,5,5.0,3.0,4.0,4.0,3.0,4.0,3.0,2.0,2,3,4,4,2.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,90.0,1,,0,,0,0.0,4.0,,1,,,,,1,2,1,2,1,2,1,2,1,2,2,2.0,자동화로 인하여 일자리감소,,철도차량용 프로그램 유지보수용,,,,3,1,39,4,전기전자,1,1,,1,42,5500,3000,,NaN,831101
4478,13970,2,4,5,5,4,5,3,3,5,4,4,4,3,5,4,5,4,5.0,4.0,3.0,4.0,3.0,3.0,4.0,5.0,4.0,4.0,2.0,3.0,3.0,4.0,2.0,2.0,5.0,4.0,3.0,3.0,1.0,1.0,4.0,4.0,1.0,1.0,3.0,4.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,2.0,1.0,3.0,2.0,4.0,4.0,2.0,6.0,2.0,,,,1,3,"기본지식,실무 훈련",6,2.0,3.0,1.0,1.0,4.0,3.0,2.0,2.0,2,3,3,3,1.0,1.0,1.0,2.0,1.0,1.0,1.0,4.0,70.0,1,,0,1,0,0.0,0.0,,1,,,,,3,,1,2,1,2,1,2,1,1,1,2.0,"자동,외주화",,"XROIS,KOVIS",설계기사,,,4,1,32,4,전자전기 공학,1,1,,1,40,2800,2800,,NaN,831101
5574,15067,1,5,5,5,5,4,3,3,3,3,3,3,5,5,5,5,2,4.0,3.0,3.0,3.0,3.0,1.0,3.0,5.0,4.0,3.0,2.0,3.0,2.0,4.0,2.0,2.0,4.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,2.0,1.0,4.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,3.0,4.0,2.0,6.0,1.0,전기기사,,,1,3,전기실무,3,4.0,3.0,3.0,3.0,4.0,4.0,4.0,3.0,3,3,4,4,1.0,1.0,2.0,2.0,1.0,1.0,2.0,3.0,80.0,1,,0,1,0,0.0,0.0,,1,,,,,1,2,1,2,1,2,1,2,1,2,2,2.0,"외주화,자동화",전기원,"엑셀,계측장비",,전기회사,,4,1,37,4,전기,1,1,,1,52,6000,3000,,NaN,831101
5784,15277,3,5,5,5,5,5,4,4,5,5,1,1,5,5,5,3,1,1.0,1.0,1.0,1.0,1.0,1.0,4.0,2.0,1.0,1.0,4.0,2.0,2.0,5.0,1.0,2.0,5.0,2.0,3.0,2.0,2.0,3.0,4.0,4.0,3.0,2.0,2.0,2.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,4.0,2.0,4.0,5.0,5.0,4.0,2.0,6.0,1.0,전기기사,,,1,6,설비관련 지식,7,5.0,4.0,4.0,4.0,5.0,4.0,5.0,3.0,4,5,5,4,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,90.0,1,,0,1,0,0.0,0.0,,1,,,,,1,2,1,2,1,2,1,2,1,1,2,3.0,정년

562101 결측치 안보임

In [43]:
meta2018[meta2018.knowcode == 562101]

,idx,cq1,cq2,cq3,cq4,cq5,cq6,cq7,cq8,cq9,cq10,cq11,cq12,cq13,cq14,cq15,cq16,cq17,cq18,cq19,cq20,cq21,cq22,cq23,cq24,cq25,cq26,cq27,cq28,cq29,cq30,cq31,cq32,cq33,cq34,cq35,cq36,cq37,cq38,cq39,cq40,cq41,cq42,cq43,cq44,cq45,cq46,cq47,cq48,cq49,cq50_1,cq50_2,cq50_3,cq50_4,cq50_5,cq50_6,cq50_7,cq50_8,iq1,iq2,iq3,iq4,iq5,iq6,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13,bq14,bq15,bq16,bq17,bq18,bq19,bq20,bq21,bq221,bq222,bq223,bq231,bq232,bq233,bq234,bq235,bq241,bq242,bq243,bq244,bq245,bq25,bq25_1,bq26_1,bq26_1a,bq26_2,bq26_2a,bq26_3,bq26_3a,bq26_4,bq26_4a,bq27,bq28,bq28_1,bq29,bq30,bq31,bq32,bq33,bq34,bq35,bq36,bq37,bq37_1,bq38,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode
120,9606,3,1,4,1,3,5,2,4,3,2,2,4,4,1,5,1,1,2.0,3.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,5.0,3.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,3.0,4.0,1.0,2.0,1,안전매니저,,,1,1,맡은지역의 요도 익히기,1.0,2.0,3.0,2.0,1.0,3.0,3.0,1.0,4,3,4,4,3.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,5.0,100,1,,0,1,0.0,0.0,0,,2,,,,,1,2,1,2,1,2,1,2,1,1,2.0,2,신축건물은 자동검침 시스템이 도입되서,안전 점검원,PDA,청호나이스 정수기 관리사원,영업직,없음,3,2,48,2,인문계,1,2,,2,40.0,,,,562101
185,9671,3,1,3,1,1,2,3,4,4,1,1,3,3,3,3,1,1,1.0,3.0,1.0,3.0,3.0,1.0,3.0,1.0,1.0,3.0,1.0,2.0,2.0,4.0,5.0,2.0,5.0,1.0,2.0,2.0,3.0,2.0,2.0,5.0,4.0,2.0,1.0,1.0,5.0,3.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,5.0,2.0,2.0,2,,,,1,1,현장실무,7.0,2.0,5.0,2.0,2.0,1.0,4.0,4.0,3,3,3,3,3.0,1.0,2.0,1.0,1.0,3.0,1.0,2.0,4.0,100,,2,0,,2.0,0.0,0,,,2,,,,1,2,2,,2,,2,,2,,1.0,1,컴퓨터화,검침원,PDA,경리사무직,없음,없음,4,2,40,3,자치행정,2,,6,,35.0,,,1000,562101
406,9892,5,3,5,2,2,4,4,5,3,1,2,3,3,4,5,1,3,2.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,4.0,3.0,2.0,2.0,2.0,1.0,3.0,4.0,4.0,1.0,3.0,1.0,4.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,3.0,4.0,4.0,3.0,4.0,1.0,3.0,2,,,,1,1,고객응대 교육,2.0,2.0,2.0,1.0,1.0,5.0,3.0,1.0,1,3,3,3,2.0,1.0,1.0,2.0,2.0,1.0,3.0,2.0,4.0,100,1,,0,1,0.0,0.0,0,,1,,,,,1,1,1,2,1,2,1,2,1,1,2.0,1,원격검침 전환,검침 매니저,"검침매니저,PDA 기계",없음,없음,없음,4,2,58,2,상과,1,1,,1,40.0,3500,3000,,562101
465,9951,3,1,4,1,3,5,3,3,2,2,2,4,3,1,5,1,1,3.0,3.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,5.0,3.0,5.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,4.0,2.0,2.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,2.0,4.0,2.0,2.0,2,,,,2,,,1.0,2.0,3.0,2.0,1.0,3.0,1.0,1.0,3,3,3,3,3.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,5.0,100,1,,0,2,0.0,0.0,0,,2,,,,,1,2,1,2,1,2,1,2,1,1,2.0,2,각종 최첨단 장비 발전으로,검침원,PDA,식당 종업원,외판원,없음,3,2,48,2,상과,1,2,,2,40.0,5000,2700,,562101
701,10187,4,3,5,3,3,5,4,4,4,2,3,3,3,2,5,1,3,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,3.0,4.0,4.0,4.0,2.0,2.0,3.0,2.0,2.0,2.0,3.0,4.0,2.0,4.0,2.0,4.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,1.0,3.0,2,,,,2,,,2.0,2.0,3.0,1.0,1.0,4.0,4.0,3.0,4,4,4,4,4.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,3.0,100,1,,0,1,0.0,0.0,0,,,2,,,,1,2,1,2,1,2,1,2,1,1,2.0,1,원격화 되어가고 있기 때문,없음,"검침용거울,망원경,후레쉬 등",없음,없음,없음,2,2,46,2,인문계,1,1,,1,45.0,3500,3000,,562101
886,10373,4,2,4,1,4,3,2,3,1,2,2,4,3,3,5,1,1,2.0,5.0,1.0,4.0,4.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,5.0,2.0,3.0,1.0,3.0,2.0,1.0,1.0,2.0,3.0,4.0,2.0,2.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,3.0,4.0,2.0,2.0,2,,,,2,,,1.0,2.0,2.0,2.0,1.0,4.0,2.0,2.0,3,3,3,4,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,4.0,100,1,,0,2,0.0,0.0,0,,2,,,,,1,2,2,,2,,2,,2,,2.0,2,컴퓨터 시스템 개발로,검침원,"볼펜, PDA",제약회사 실험실 보조직원,우유 배달,없음,3,2,51,2,상과,1,2,,2,40.0,3000,1800,,562101
1146,10634,3,1,4,1,3,5,2,4,2,2,2,3,3,1,5,1,1,1.0,3.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,5.0,3.0,3.0,1.0,2.0,2.0,1.0,2.0,2.0,4.0,3.0,2.0,2.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,2.0,2.0,3.0,4.0,1.0,2.0,1,가스안전 매니저,,,2,,,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,4,3,3,4,4.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,5.0,100,1,,0,1,0.0,0.0,0,,2,,,,,1,2,1,2,1,2,1,2,1,1,2.0,2,최신 첨단 기술 발전으로 원격 자동검침기능이 발전해서,검침원,PDA,매장점원,없음,없음,2,2,49,

KnowCode: 415404
- Index 3064 : bq14부터 오른쪽 한칸이동
- Index 8551 : bq33부터 오른쪽 한칸이동

In [44]:
meta2018[meta2018.knowcode == 415404]

,idx,cq1,cq2,cq3,cq4,cq5,cq6,cq7,cq8,cq9,cq10,cq11,cq12,cq13,cq14,cq15,cq16,cq17,cq18,cq19,cq20,cq21,cq22,cq23,cq24,cq25,cq26,cq27,cq28,cq29,cq30,cq31,cq32,cq33,cq34,cq35,cq36,cq37,cq38,cq39,cq40,cq41,cq42,cq43,cq44,cq45,cq46,cq47,cq48,cq49,cq50_1,cq50_2,cq50_3,cq50_4,cq50_5,cq50_6,cq50_7,cq50_8,iq1,iq2,iq3,iq4,iq5,iq6,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13,bq14,bq15,bq16,bq17,bq18,bq19,bq20,bq21,bq221,bq222,bq223,bq231,bq232,bq233,bq234,bq235,bq241,bq242,bq243,bq244,bq245,bq25,bq25_1,bq26_1,bq26_1a,bq26_2,bq26_2a,bq26_3,bq26_3a,bq26_4,bq26_4a,bq27,bq28,bq28_1,bq29,bq30,bq31,bq32,bq33,bq34,bq35,bq36,bq37,bq37_1,bq38,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode
1420,10908,4,2,4,5,3,4,5,4,4,4,5,3,3,4,3,4,3,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,2.0,1.0,5.0,4.0,4.0,5.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0,1.0,2.0,4.0,1.0,2.0,3.0,2.0,1.0,1.0,1.0,2.0,4.0,5.0,5.0,5.0,5.0,4.0,13.0,1.0,2.0,2,,,,2,,,5.0,5.0,4.0,2.0,2.0,4.0,5.0,2.0,3,,5,5,4.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,80,,2,0,,0.0,0.0,0,5,1,2,,,,3,,1,3,1,3,2,,1,3,2.0,1,광고시장이 줄어들기 때문이다,직급을 부른다.,일러스트 포토샵,디자이너,마케팅전문가,제품개발을 하여 프렌차이점,1,1,56,6,디자인학과,2,,6,,55,,,2000,415404
1468,10956,4,4,4,4,4,4,4,4,4,3,3,3,3,4,4,4,3,3.0,2.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,2.0,2.0,1.0,5.0,1.0,5.0,5.0,4.0,3.0,2.0,5.0,4.0,3.0,5.0,3.0,4.0,2.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,3.0,5.0,5.0,5.0,3.0,3.0,18.0,1.0,2.0,1,시각디자인산업기사,,,2,,,3.0,5.0,4.0,4.0,4.0,4.0,4.0,2.0,3,3,3,3,3.0,2.0,2.0,3.0,2.0,3.0,2.0,2.0,4.0,100,1,,0,1,0.0,0.0,0,,1,,,,,1,2,1,2,1,2,1,2,1,2,1.0,2,다양하고 수많은 광고매체를 통해 광고를 디자인하므로 경쟁률이 높다,아트커뮤니터,포토샵,,,,3,2,42,4,산업디자인,1,1,,1,60,3800,1800,,415404
1658,11146,5,3,3,3,3,3,4,3,4,1,2,3,3,5,1,1,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,4.0,1.0,2.0,1.0,3.0,3.0,4.0,3.0,2.0,2.0,2.0,4.0,4.0,1.0,5.0,1.0,2.0,4.0,1.0,3.0,1.0,3.0,1.0,3.0,1.0,2.0,2.0,4.0,5.0,3.0,5.0,4.0,13.0,1.0,2.0,2,,,,1,5,컬러이해,4.0,4.0,4.0,2.0,2.0,3.0,5.0,1.0,2,5,4,4,4.0,3.0,2.0,1.0,2.0,1.0,1.0,2.0,4.0,90,1,,0,,0.0,0.0,4,,,2,,,,3,,1,2,1,2,1,2,1,2,1.0,2,자동화,없음,포토샵,없음,없음,없음,5,2,29,4,컴퓨터그래픽,1,2,,1,60,3200,2000,,415404
1807,11295,4,3,4,5,2,3,4,1,3,2,2,2,2,5,2,1,3,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,5.0,1.0,1.0,1.0,1.0,5.0,1.0,2.0,3.0,3.0,2.0,2.0,3.0,3.0,3.0,2.0,1.0,3.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,4.0,3.0,5.0,4.0,4.0,3.0,3.0,1.0,4.0,2,,,,2,,,4.0,5.0,3.0,5.0,5.0,3.0,3.0,4.0,3,2,3,3,3.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,90,,,3,1,0.0,0.0,0,,1,,,,,1,2,1,2,1,2,1,2,1,1,2.0,3,꾸준한소비가 있는업체,없음,"일러스트, 포토샵",없음,없음,안전확인업체,4,2,30,4,시각디자인,1,1,,1,40,6000,4000,,415404
2271,11761,5,3,5,5,5,5,4,4,4,3,3,4,2,5,2,1,3,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,2.0,5.0,3.0,2.0,1.0,2.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,5.0,5.0,3.0,3.0,4.0,10.0,1.0,4.0,1,웹디자인기능사,컴퓨터그래픽,,1,2,"툴 능력, 기획 과정",2.0,2.0,4.0,4.0,4.0,3.0,4.0,3.0,3,3,4,4,4.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,90,1,,0,1,0.0,0.0,0,,1,,,,,1,2,1,2,1,2,1,2,1,2,1.0,2,인쇄물 감소,크리에이터,포토샵 일러스트,없음,마케터,크리에이터,3,2,29,4,멀티미디어,1,1,,1,40,3120,2280,,415404
2347,11837,5,3,5,5,5,5,5,5,5,2,3,5,4,5,3,3,3,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,2.0,2.0,3.0,2.0,5.0,1.0,3.0,3.0,2.0,1.0,2.0,4.0,5.0,2.0,2.0,1.0,4.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,5.0,5.0,2.0,5.0,5.0,5.0,13.0,1.0,3.0,2,,,,2,,,5.0,5.0,3.0,1.0,1.0,3.0,4.0,1.0,3,3,3,3,3.0,2.0,1.0,1.0,2.0,3.0,1.0,1.0,3.0,90,1,,0,1,0.0,0.0,0,,1,,,,,1,2,1,2,1,2,1,2,1,1,2.0,2,AI와 기술발전,없음,광고관리프로그램,없음,광고대행사AE,없음,1,1,27,4,영어영문,1,1,,1,52,2600,2500,,415404
2685,12175,3,3,5,5,3,5,2,5,4,1,3,3,2,5,2,1,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,2.0,1.0,3.0,2.0,2.0,1.0,2.0,2.0,4.0,2.0,2.0,5.0,4.0,2.0,2.0,1.0,2.0,1.0,1.0,3.0,1.0,2.0,2.0,3.0,1.0,1.0,1.0,1.0,3.0,5.0,5.0,5.0,1.0,2.0,16.0,1.0,1.0,1,컬러리스트,컬러테라피스트,퍼스널컬러컨설턴트,1,4,컨설팅 업무를 위한 서비스 훈련,4.0,4.0,4.0,3.0,1.0,4.0,5.0,4.0,1,3,3,3,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,3.0,100,1,,0,1,0.0,0.0,0,,1,,,,,1,1,2,,2,,2,,2,,2.0,3,발

Knowcode : 622304

Index 8205 : bq30부터 오른쪽으로 4칸 이동후 bq30~33은 결측으로 둠

In [45]:
meta2018[meta2018.knowcode == 622304]

,idx,cq1,cq2,cq3,cq4,cq5,cq6,cq7,cq8,cq9,cq10,cq11,cq12,cq13,cq14,cq15,cq16,cq17,cq18,cq19,cq20,cq21,cq22,cq23,cq24,cq25,cq26,cq27,cq28,cq29,cq30,cq31,cq32,cq33,cq34,cq35,cq36,cq37,cq38,cq39,cq40,cq41,cq42,cq43,cq44,cq45,cq46,cq47,cq48,cq49,cq50_1,cq50_2,cq50_3,cq50_4,cq50_5,cq50_6,cq50_7,cq50_8,iq1,iq2,iq3,iq4,iq5,iq6,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13,bq14,bq15,bq16,bq17,bq18,bq19,bq20,bq21,bq221,bq222,bq223,bq231,bq232,bq233,bq234,bq235,bq241,bq242,bq243,bq244,bq245,bq25,bq25_1,bq26_1,bq26_1a,bq26_2,bq26_2a,bq26_3,bq26_3a,bq26_4,bq26_4a,bq27,bq28,bq28_1,bq29,bq30,bq31,bq32,bq33,bq34,bq35,bq36,bq37,bq37_1,bq38,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode
330,9816,5,1,5,1,1,4,1,4,1,1,1,2,3,1,5,5,4,3.0,2.0,1.0,2.0,4.0,2.0,1.0,1.0,2.0,4.0,2.0,5.0,1.0,2.0,3.0,2.0,5.0,4.0,3.0,2.0,2.0,4.0,1.0,4.0,4.0,4.0,3.0,3.0,3.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,3.0,2.0,3.0,8.0,4.0,1.0,1,1종대형면허증,,,2,,,6.0,2.0,3.0,3.0,1.0,3.0,3.0,2.0,3,,,,3.0,3.0,2.0,4.0,1.0,3.0,3.0,3.0,4.0,0,,2,0,,0.0,0.0,4,,,,,4,,3,,2,,2,,2,,2,,2.0,4,타업종 보다 본인의사대로 일할수 있는 직종으로,,차,영업택시기사,,,3,1,42,2,인문계,2,,6,,50.0,,,3800,622304
532,10018,5,1,5,1,1,5,4,1,1,3,4,1,1,1,5,5,1,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,5.0,1.0,5.0,3.0,1.0,1.0,4.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,8.0,4.0,1.0,1,1종면허,,,2,,,2.0,2.0,2.0,1.0,1.0,4.0,3.0,3.0,2,,,,4.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,4.0,80,,2,0,,0.0,0.0,0,5,,,,,5,2,,2,,2,,1,1,1,1,2.0,2,경기침체,,레미콘차량,냉동차 운전사업,,,1,1,39,2,문과,2,,6,,48.0,,,1800,622304
715,10201,3,2,5,1,1,3,1,2,2,1,1,1,3,1,4,5,1,2.0,3.0,1.0,4.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,3.0,3.0,1.0,3.0,2.0,4.0,4.0,4.0,3.0,5.0,3.0,4.0,3.0,5.0,1.0,3.0,4.0,2.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,3.0,8.0,1.0,2.0,1,특수1종 대형면허,,,1,3,길을 찾는요령 및 주유방법,4.0,2.0,2.0,3.0,2.0,4.0,3.0,4.0,4,4,4,4,4.0,3.0,2.0,2.0,1.0,3.0,3.0,3.0,3.0,0,,2,0,,0.0,0.0,0,5,,,,,5,1,1,3,,3,,3,,3,,2.0,1,"전기차,수소차 등장",기사님,차,축구선수,,,4,1,36,2,문,2,,6,,30.0,,,3600,622304
925,10413,5,1,4,2,2,4,4,4,4,1,2,1,1,1,5,1,3,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,3.0,2.0,2.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,5.0,1.0,4.0,4.0,5.0,8.0,1.0,2.0,1,1종보통운전면허,,,2,,,2.0,2.0,2.0,3.0,2.0,4.0,3.0,3.0,4,4,4,4,4.0,3.0,1.0,2.0,1.0,3.0,2.0,3.0,5.0,100,,2,0,,2.0,0.0,0,,1,,,,,1,2,1,3,1,3,1,3,1,3,1.0,2,경기침체때문에,"운전(기사),선생님","트럭,화물,기타",서비스직,,,2,1,50,2,,2,,6,,40.0,,,3300,622304
958,10446,5,1,3,1,1,5,2,4,4,3,3,3,1,2,5,5,2,4.0,4.0,2.0,2.0,4.0,3.0,2.0,2.0,4.0,4.0,1.0,3.0,3.0,1.0,3.0,1.0,4.0,2.0,4.0,4.0,3.0,3.0,3.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,4.0,3.0,3.0,3.0,3.0,3.0,8.0,1.0,2.0,1,1종대형면허증,,,2,,,1.0,1.0,4.0,4.0,3.0,3.0,2.0,3.0,2,2,2,3,3.0,2.0,2.0,3.0,2.0,2.0,3.0,1.0,3.0,0,1,,0,1,0.0,0.0,0,,1,,,,,1,1,1,2,1,2,2,,2,,1.0,2,노동댓가 적어서,기사님,덤프트럭,인쇄사 기술직,,,2,1,57,1,,1,1,,1,48.0,4000,3000,,622304
1209,10697,5,1,4,1,1,4,4,4,3,1,1,2,2,1,5,5,1,5.0,4.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,5.0,1.0,1.0,1.0,2.0,5.0,1.0,5.0,3.0,1.0,2.0,4.0,4.0,3.0,3.0,1.0,2.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,1,1종대형면허,,,1,1,현장업무 교육,3.0,1.0,3.0,3.0,3.0,4.0,2.0,1.0,3,,,3,3.0,3.0,2.0,1.0,2.0,1.0,3.0,1.0,2.0,0,,2,0,,0.0,0.0,0,5,,,,,5,1,2,2,,2,,2,,2,,2.0,3,건설업이 크게 증가하거나 감소하지 않을것같다,레미콘 기사,레미콘 차량,자영업,없음,없음,2,1,56,4,세무학,1,1,,1,60.0,3000,,,622304
1291,10779,4,1,3,1,1,3,3,3,1,1,1,2,2,1,5,5,4,5.0,2.0,1.0,5.0,5.0,3.0,2.0,1.0,2.0,5.0,3.0,5.0,2.0,1.0,4.0,3.0,5.0,4.0,2.0,1.0,1.0,5.0,1.0,2.0,2.0,4.0,2.0,3.0,1.0,2.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,2.0,2.0,2.0,4.0,1.0,8.0,4.0,1.0,1,1종 대형면허,,,2,,,5.0,2.0,4.0,3.0,2.0,4.0,2.0,2.0,2,,,,3.0,2.0,2.0,4.0,1.0,3.0,3.0,3.0,4.0,0,,2,0,,0.0,3.0,0,,,,,4,,3,,2,,2,,2,,2,,2.0,2,물동량 감소로 수입이 적으므로,,내비게이션,택시운전,물류회사 운영 CEO,,3,1,60,1,,2,,6,,45.0,,,3800,622304
1358,10846,3,1,4,1,2,4,1,4,2,1,1,2,3,1,5,5,2,2

Knowcode : 131201

Index 8205 : bq33부터 오른쪽으로 한칸씩

In [46]:
meta2018[meta2018.knowcode == 131201]

,idx,cq1,cq2,cq3,cq4,cq5,cq6,cq7,cq8,cq9,cq10,cq11,cq12,cq13,cq14,cq15,cq16,cq17,cq18,cq19,cq20,cq21,cq22,cq23,cq24,cq25,cq26,cq27,cq28,cq29,cq30,cq31,cq32,cq33,cq34,cq35,cq36,cq37,cq38,cq39,cq40,cq41,cq42,cq43,cq44,cq45,cq46,cq47,cq48,cq49,cq50_1,cq50_2,cq50_3,cq50_4,cq50_5,cq50_6,cq50_7,cq50_8,iq1,iq2,iq3,iq4,iq5,iq6,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13,bq14,bq15,bq16,bq17,bq18,bq19,bq20,bq21,bq221,bq222,bq223,bq231,bq232,bq233,bq234,bq235,bq241,bq242,bq243,bq244,bq245,bq25,bq25_1,bq26_1,bq26_1a,bq26_2,bq26_2a,bq26_3,bq26_3a,bq26_4,bq26_4a,bq27,bq28,bq28_1,bq29,bq30,bq31,bq32,bq33,bq34,bq35,bq36,bq37,bq37_1,bq38,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode
552,10038,5,2,3,3,2,2,5,3,2,1,3,2,2,5,1,1,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,1.0,3.0,1.0,3.0,3.0,3.0,2.0,2.0,1.0,3.0,4.0,3.0,2.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,5.0,3.0,1.0,2.0,2.0,4.0,3.0,1.0,3.0,2,,,,2,,,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3,3,3,3,3.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,100,,,3,1,0.0,0.0,0,,1,,,,,1,2,1,2,1,2,1,2,1,1,1.0,3,시험하는 일은 꾸준히 계속된다,없음,시험기계,없음,없음,없음,3,2,24,3,전자통신,1,1,,1,52,2500,1600,,131201
556,10042,4,4,4,4,4,2,3,3,4,4,4,3,1,5,1,1,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,2.0,1.0,5.0,5.0,4.0,1.0,4.0,5.0,5.0,1.0,2.0,4.0,5.0,1.0,1.0,4.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,5.0,5.0,1.0,2.0,4.0,3.0,13.0,1.0,3.0,1,사무자동화산업기사,,,1,1,무선설비 다루기 훈련,2.0,4.0,4.0,3.0,3.0,4.0,4.0,4.0,3,3,3,3,3.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,4.0,70,,,3,1,0.0,0.0,0,,1,,,,,1,2,1,2,1,2,1,2,1,1,2.0,3,통신수요만큼 공급도 포화상태이므로,,내부프로그램 사무용품,,,,2,1,31,4,정보통신,1,1,,1,40,3000,2400,,131201
762,10248,5,3,5,5,4,5,3,3,3,2,2,4,3,5,3,4,3,4.0,4.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0,2.0,2.0,5.0,2.0,1.0,2.0,2.0,2.0,3.0,1.0,2.0,2.0,3.0,2.0,4.0,3.0,3.0,2.0,3.0,4.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,3.0,3.0,2.0,2.0,3.0,4.0,13.0,2.0,5.0,2,,,,1,2,"시험기기 다루는 법, 설계 도면 보는 방법 등",4.0,4.0,3.0,3.0,3.0,4.0,4.0,3.0,3,2,3,3,3.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,95,1,,0,1,0.0,0.0,0,,,2,,,,1,2,1,2,1,2,1,2,1,1,2.0,3,통신의 기술은 계속된다,없음,"캐드, 시험기기",없음,"전자, 전기 기술자",없음,3,2,36,4,영문학과,1,1,,1,40,4500,3900,,131201
855,10342,5,3,3,3,3,2,5,3,2,1,2,3,2,5,1,1,3,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,3.0,3.0,2.0,3.0,2.0,3.0,4.0,3.0,4.0,3.0,1.0,3.0,4.0,3.0,1.0,2.0,3.0,3.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,3.0,2.0,5.0,3.0,1.0,3.0,2,,,,2,,,7.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3,3,3,3,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,100,1,,0,1,0.0,0.0,0,,1,,,,,1,2,1,2,1,2,1,2,1,1,1.0,3,변화없을 예정이라서,없음,시험기기,없음,전자기기 시험원,없음,3,1,29,4,컴퓨터공학,1,1,,1,40,2450,1900,,131201
2247,11737,4,1,5,5,5,5,3,4,3,2,2,2,2,5,4,4,3,4.0,4.0,4.0,4.0,3.0,1.0,1.0,4.0,1.0,1.0,1.0,5.0,2.0,1.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,3.0,4.0,4.0,4.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,1.0,3.0,3.0,2.0,10.0,1.0,6.0,2,,,,1,1,사내 업무파악,6.0,5.0,4.0,4.0,3.0,3.0,3.0,3.0,3,3,3,3,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,100,1,,0,1,0.0,0.0,0,,,,,,5,1,2,1,2,1,2,1,2,1,1,2.0,1,장비의 자동화,,통신용계측기,,,,3,1,58,4,전자통신,1,1,,1,42,6000,4200,,131201
2589,12079,5,1,3,3,3,3,5,3,2,2,2,3,3,5,3,3,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,3.0,3.0,2.0,3.0,2.0,3.0,4.0,4.0,4.0,3.0,2.0,2.0,5.0,3.0,2.0,2.0,3.0,3.0,1.0,1.0,3.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,3.0,4.0,1.0,2.0,3.0,3.0,3.0,1.0,3.0,1,AUTO CAD,,,1,2,설계관련 교육,7.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3,3,3,3,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,100,1,,0,1,0.0,0.0,0,,1,,,,,1,2,1,2,1,2,1,2,1,1,1.0,3,기계나 장비를 만들려면 시험을 꼭 해야 되는 곳이리 때문에,없음,AUTO CAD,없음,없음,없음,3,1,24,3,전자컴퓨터통신,1,1,,1,52,2850,1600,,131201
3340,12831,4,4,4,5,3,4,4,3,4,2,3,2,3,5,2,1,3,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,4.0,2.0,1.0,1.0,1.0,4.0,1.0,4.0,4.0,3.0,3.0,4.0,4.0,3.0,3.0,3.0,2.0,2.0,1.0,1.0,2.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,3.0,4.0,2.0,3.0,1.0,2.0,1,정보통신기사,,,2,,,3.0,4.0,4.0,3.0,3.0,4.0,4.0,3.0,5,4,4,4,4.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,90,1,,0,1,0.0,0.0,0,,1,,,,,1,2,1,2,1,2,1,2,1,1,1.0,3,현재 통신기기포화상태라서,,"자

2019년 : 2018년과 같은 방식으로 진행

In [47]:
c_list = meta2019.columns.tolist()
nan2019 = []

for i in c_list:
    nan2019.append([i,list(meta2019[meta2019[i].isnull()].index)])
nan2019

[['idx', []],
 ['sq1', []],
 ['sq2', []],
 ['sq3', []],
 ['sq4', []],
 ['sq5', []],
 ['sq6', []],
 ['sq7', []],
 ['sq8', []],
 ['sq9', []],
 ['sq10', []],
 ['sq11', []],
 ['sq12', []],
 ['sq13', []],
 ['sq14', []],
 ['sq15', []],
 ['sq16', []],
 ['kq1_1', []],
 ['kq1_2', []],
 ['kq2_1', []],
 ['kq2_2', []],
 ['kq3_1', []],
 ['kq3_2', []],
 ['kq4_1', []],
 ['kq4_2', []],
 ['kq5_1', []],
 ['kq5_2', []],
 ['kq6_1', []],
 ['kq6_2', []],
 ['kq7_1', []],
 ['kq7_2', []],
 ['kq8_1', []],
 ['kq8_2', []],
 ['kq9_1', []],
 ['kq9_2', []],
 ['kq10_1', []],
 ['kq10_2', []],
 ['kq11_1', []],
 ['kq11_2', []],
 ['kq12_1', []],
 ['kq12_2', []],
 ['kq13_1', []],
 ['kq13_2', []],
 ['kq14_1', []],
 ['kq14_2', []],
 ['kq15_1', []],
 ['kq15_2', []],
 ['kq16_1', []],
 ['kq16_2', []],
 ['kq17_1', []],
 ['kq17_2', []],
 ['kq18_1', []],
 ['kq18_2', []],
 ['kq19_1', []],
 ['kq19_2', []],
 ['kq20_1', []],
 ['kq20_2', []],
 ['kq21_1', []],
 ['kq21_2', []],
 ['kq22_1', []],
 ['kq22_2', []],
 ['kq23_1', []],
 ['kq23_

Nan 값 있는 컬럼들

In [48]:
meta2019.loc[[177, 614, 1394, 2610, 2687, 3309, 3361, 7251, 7758]]['knowcode'].value_counts()

13902     2
414303    1
158501    1
134303    1
415503    1
413202    1
132002    1
151201    1
Name: knowcode, dtype: int64

Knowcode : 13902

Index 1394 : bq6과 bq7을 bq20_1로 밀어냄
bq8부터 bq21로 밀어냄

In [49]:
meta2019[meta2019.knowcode == 13902]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,kq6_1,kq6_2,kq7_1,kq7_2,kq8_1,kq8_2,kq9_1,kq9_2,kq10_1,kq10_2,kq11_1,kq11_2,kq12_1,kq12_2,kq13_1,kq13_2,kq14_1,kq14_2,kq15_1,kq15_2,kq16_1,kq16_2,kq17_1,kq17_2,kq18_1,kq18_2,kq19_1,kq19_2,kq20_1,kq20_2,kq21_1,kq21_2,kq22_1,kq22_2,kq23_1,kq23_2,kq24_1,kq24_2,kq25_1,kq25_2,kq26_1,kq26_2,kq27_1,kq27_2,kq28_1,kq28_2,kq29_1,kq29_2,kq30_1,kq30_2,kq31_1,kq31_2,kq32_1,kq32_2,kq33_1,kq33_2,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13_1,bq13_2,bq13_3,bq14_1,bq14_2,bq14_3,bq14_4,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq22,bq23,bq24,bq25,bq26,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
43,18612,4,4,4,4,4,3,4,3,5,4,4,3,4,3,3,4,3,4,2,3,3,5,3,5,3,5,2,5,1,0,1,0,3,5,1,0,4,6,1,0,1,0,4,6,1,0,1,0,1,0,4,5,4,6,3,4,1,0,4,6,4,6,4,5,3,4,1,0,1,0,3,4,3,4,3,4,2,4,3,5,2,3,13,1,2,2,,,,1,6,,6,6,5,2,2,4,4,2,3,9,9,9,3,4.0,2,3.0,2.0,2.0,2.0,1,1.0,3.0,4.0,3.0,4.0,1.0,1.0,5.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,3.0,2,,1.0,4.0,4,,,,1,56.0,4,,1.0,1,,1,50.0,,,,13902
127,18696,4,5,5,3,5,5,3,5,5,5,5,5,5,5,5,5,3,4,3,4,3,4,3,5,5,5,3,5,1,0,1,0,2,4,1,0,2,3,1,0,1,0,2,5,1,0,1,0,1,0,3,5,2,3,1,0,1,0,2,3,2,4,4,4,4,5,1,0,1,0,1,0,4,5,3,3,1,0,4,4,1,0,13,1,5,2,,,,1,4,"통계, 회사내 솔루션 및 모델",7,5,2,1,1,4,4,1,2,2,2,2,2,5.0,1,1.0,1.0,2.0,2.0,1,1.0,5.0,5.0,5.0,5.0,1.0,2.0,5.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,여러 직무에 대한 업무 추가,5.0,1,기술의 진보로 기술적인 업무 뿐만 아니라 인사이트까지 기술로 대체될 수 있다,1.0,1.0,4,제조업 사무직,,B8ta,1,47.0,5,MBA,1.0,1,,1,50.0,10000,3500,,13902
262,18831,5,4,5,5,4,4,4,4,4,4,5,4,4,4,5,5,5,6,3,3,4,5,5,5,5,5,5,5,1,0,1,0,3,4,1,0,1,0,1,0,1,0,3,3,1,0,1,0,1,0,4,5,3,4,1,0,1,0,3,4,2,2,3,4,2,2,1,0,1,0,1,0,3,4,4,5,1,0,4,5,1,0,13,1,2,2,,,,2,,,7,5,3,3,2,3,3,3,3,9,9,9,3,3.0,3,2.0,3.0,3.0,3.0,2,3.0,4.0,4.0,2.0,4.0,2.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,1.0,3,,2.0,2.0,2,광고회사,,,1,55.0,5,경영학,1.0,1,,1,50.0,10000,10000,,13902
634,19203,4,5,4,5,4,4,4,5,5,4,5,5,5,4,5,5,2,2,2,4,1,0,2,4,4,6,2,4,1,0,1,0,1,0,1,0,1,0,1,0,1,0,4,4,1,0,1,0,1,0,4,3,2,3,1,0,1,0,2,4,2,4,3,4,2,5,1,0,1,0,1,0,1,0,1,0,1,0,2,4,1,0,13,1,3,2,,,,2,,,6,5,4,4,4,4,4,3,4,3,4,3,4,1.0,4,4.0,5.0,5.0,5.0,4,5.0,3.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,1.0,3,데이터를 비교하고 분석하는 작업은 프로그램으로 할수있지만 의미를 해석하는 분야는 여...,1.0,3.0,3,공공기관근무,"교육강사, 모더레이터",,1,48.0,5,산업공학과,1.0,1,,1,40.0,10000,6000,,13902
748,19317,4,4,5,4,4,5,3,4,3,4,5,5,4,3,3,5,4,4,5,5,2,3,3,4,4,5,3,4,2,3,1,0,4,4,2,2,3,3,1,0,1,0,3,4,1,0,1,0,1,0,4,5,3,5,1,0,1,0,1,0,3,6,5,6,3,5,1,0,2,5,3,5,1,0,3,4,1,0,3,5,2,4,13,1,4,2,,,,1,2,통계 패키지 활용 및 분석,6,5,4,4,3,5,5,4,4,4,4,5,5,1.0,5,4.0,5.0,6.0,5.0,4,5.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,새로운 통계 패키지 활용 시도로 그에 대한 교육및 이용 증가,2.0,3,AI등의 발달로 새로운 시스템이 도입될것이나 시장의 수요도 늘어날 것이다,1.0,1.0,3,,,,1,50.0,4,심리학,1.0,1,,1,40.0,13000,4000,,13902
1394,19963,3,4,4,4,5,4,5,5,5,4,5,5,3,4,3,4,3,4,3,4,3,4,4,6,3,5,3,5,2,3,1,0,2,4,1,0,1,0,1,0,1,0,2,5,1,0,1,0,1,0,2,4,2,4,1,0,1,0,2,4,2,4,3,5,2,5,1,0,1,0,1,0,1,0,2,4,2,4,2,4,1,0,13,1,4,2,,,,1,4,조사 방법이나 통계에 대한 교육 보다는 시장/세상에 대한 이해가 필요하다고 생각됨_...,기존 사람들이 진행하던 분석 업무들 중 일부도 대체 가능해 보임_다만,경험을 기준으로 하는 분석이나 새로운 아이디어 부분에서는 여전히 사람이 진행해야 ...,2,4,2,,잘 떠오르지는 않으나 회사 생활을 오래 해왔기 때문에 대부분의 사무 업무는 가능할 ...,모바일 혹은 GPS 기술 등을 접목한 서베이 기술 관련 프로그래머/기술자??,2,44,5,사회학과,1,3.0,,1.0,50.0,8300.0,3200.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13902
4213,22783,4,4,5,4,3,4,5,4,5,4,5,3,5,3,4,5,3,4,3,5,3,4,4,6,4,5,4,5,2,4,1,0,3,4,2,4,3,5,1,0,1,0,5,7,2,3,1,0,1,0,4,6,4,6,3,5,1,0,4,6,4,6,4,6,2,4,1,0,3,4,2,3,4,5,4,5,3,4,4,5,2,3,13,1,3,2,,,,1,1,통계 교육,6,6,4,3,3,3,3,4,3,5,4,3,4,3.0,4,4.0,4.0,4.0,3.0,5,3.0,2.0,4.0,2.0,2.0,2.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,2.0,2

Knowcode : 13902

Index 7758 : bq20부터 한 칸 뒤로

In [50]:
meta2019[meta2019.knowcode == 414303]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,kq6_1,kq6_2,kq7_1,kq7_2,kq8_1,kq8_2,kq9_1,kq9_2,kq10_1,kq10_2,kq11_1,kq11_2,kq12_1,kq12_2,kq13_1,kq13_2,kq14_1,kq14_2,kq15_1,kq15_2,kq16_1,kq16_2,kq17_1,kq17_2,kq18_1,kq18_2,kq19_1,kq19_2,kq20_1,kq20_2,kq21_1,kq21_2,kq22_1,kq22_2,kq23_1,kq23_2,kq24_1,kq24_2,kq25_1,kq25_2,kq26_1,kq26_2,kq27_1,kq27_2,kq28_1,kq28_2,kq29_1,kq29_2,kq30_1,kq30_2,kq31_1,kq31_2,kq32_1,kq32_2,kq33_1,kq33_2,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13_1,bq13_2,bq13_3,bq14_1,bq14_2,bq14_3,bq14_4,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq22,bq23,bq24,bq25,bq26,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
1065,19634,3,4,4,2,4,4,3,3,4,4,3,4,2,3,4,3,1,0,3,3,1,0,2,4,3,3,1,0,1,0,2,3,2,3,1,0,4,6,2,3,1,0,2,3,2,3,1,0,1,0,3,4,3,4,2,3,1,0,2,3,3,4,3,3,2,3,4,4,2,3,2,3,1,0,1,0,2,3,4,4,1,0,10,1,2,2,,,,1,3,"필요한 툴다루기, 각회사별 작품 연출방식 익히기",5,3,4,2,2,4,4,3,4,4,3,4,4,2.0,2,3.0,4.0,4.0,3.0,4,2.0,3.0,4.0,3.0,3.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,사용하던 툴 교체,2.0,2,3D가 많이 발달되서,1.0,2.0,4,,,,2,29.0,4,만화 애니메이션과,1.0,1,,1,40,3000,1500,,414303
1141,19710,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,2,3,2,3,2,3,2,3,2,3,2,3,2,3,2,3,2,3,2,3,3,4,2,3,1,0,2,3,1,0,1,0,1,0,2,3,2,3,1,0,1,0,2,2,2,2,2,3,2,1,3,4,2,2,1,0,3,4,3,4,2,3,3,4,1,0,10,1,2,2,,,,1,4,,3,3,4,3,3,4,4,4,3,3,4,4,4,3.0,3,3.0,5.0,5.0,5.0,5,5.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,2.0,2,,2.0,2.0,2,,,,2,41.0,2,특성화고,1.0,1,,1,40,2400,2100,,414303
1333,19902,4,5,5,5,5,5,5,4,5,5,4,4,4,3,5,3,3,4,2,4,1,0,5,6,3,4,2,4,1,0,1,0,4,5,1,0,5,6,1,0,3,3,1,0,2,3,1,0,1,0,3,4,1,0,2,2,1,0,1,0,4,4,5,5,4,5,5,5,3,4,1,0,1,0,1,0,2,4,5,5,1,0,10,1,2,2,,,,2,,,3,4,4,4,3,3,5,3,5,5,5,5,5,2.0,4,4.0,5.0,5.0,5.0,5,5.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,1.0,3,심미적인 감각은 대체될수 없다고 생각함,2.0,4.0,3,,,,1,35.0,4,전자과,1.0,1,,1,40,4200,2500,,414303
1350,19919,2,4,4,3,4,3,4,3,5,3,3,4,2,2,1,2,2,4,2,5,2,3,1,0,1,0,1,0,1,0,1,0,3,5,1,0,3,3,1,0,2,1,2,3,1,0,1,0,1,0,2,3,3,4,1,0,1,0,1,0,1,0,1,0,1,0,3,4,2,3,3,3,1,0,2,2,3,5,3,5,2,2,10,1,2,2,,,,2,,,3,4,3,3,2,4,4,2,1,1,1,3,1,2.0,2,2.0,2.0,2.0,2.0,3,3.0,3.0,2.0,3.0,3.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,1.0,3,사람이 만지기는 해야함,2.0,3.0,3,,,,2,29.0,4,시각디자인학과,1.0,1,,1,50,2700,1800,,414303
1675,20245,4,3,3,3,5,3,3,4,4,3,3,4,3,3,2,3,2,2,2,2,2,2,2,2,2,2,1,0,2,2,1,0,1,0,1,0,3,5,1,0,1,0,1,0,1,0,1,0,1,0,2,2,1,0,1,0,1,0,2,3,2,3,1,0,1,0,2,3,2,3,1,0,1,0,1,0,1,0,2,3,1,0,10,1,2,2,,,,1,5,,5,1,3,2,2,3,3,2,4,3,3,3,3,2.0,4,3.0,3.0,3.0,3.0,3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,기존에 종이에 수작업으로 하던것을 디지털로 전환하여 하게 되는 부분들이 생겼다._애...,2.0,4,이쪽 일을 하시는 분들의 감소로 인해 일자리는 더더욱 증가할거 같습니다.,2.0,2.0,2,,,,1,51.0,2,멀티,1.0,1,,1,40,3600,2400,,414303
2002,20572,4,4,3,2,4,5,3,4,4,5,5,5,4,3,4,5,3,1,3,3,1,0,2,1,2,2,1,0,1,0,1,0,4,5,4,4,5,4,1,0,1,0,3,4,2,5,1,0,3,4,4,4,4,3,3,3,1,0,1,0,1,0,2,3,2,4,5,5,3,4,1,0,1,0,1,0,3,5,5,6,1,0,10,1,2,2,,,,1,6,애니메이션 교육,7,5,3,2,1,4,4,2,2,2,4,4,3,2.0,4,4.0,2.0,2.0,2.0,2,3.0,2.0,4.0,3.0,3.0,3.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,신기술과 장비의 도입,1.0,3,대체제 없음,2.0,2.0,2,,연출,모션캡쳐배우,1,28.0,4,미디어영상학과,1.0,3,,2,52,2500,1800,,414303
2079,20649,4,5,4,3,4,2,2,5,5,5,5,5,4,5,4,3,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,2,5,1,0,2,3,1,0,1,0,2,2,1,0,1,0,1,0,3,3,2,2,1,0,1,0,1,0,1,0,1,0,1,0,3,4,1,0,1,0,2,2,2,3,1,0,4,6,1,0,10,1,2,2,,,,1,2,"애니메이팅의 기본,인체",5,2,4,3,2,3,4,2,3,3,3,4,3,3.0,3,3.0,3.0,3.0,3.0,2,2.0,3.0,2.0,3.0,3.0,2.0,2.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,1.0,3,"현재도 작은 시작이며,기계가 발달함으로 인해 대체될 직업은 아니라는생각",2.0,3.0,3,,"(3D,2D,게임)애니메이터,디자이너,원화가,일러스트레이터",,2,23.0,4,애니메이션과,1.0,1,,1,45,2400,2350,,414303
2687,21257,3,2,2,3,3,2,3,2,2,3,3,2,2,2,3,3,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,2,4,1,0,3,5,1,0,1,0,1,0,1,0,1,0,1,0,2,3,2,2,1,

Knowcode : 158501

Index 3361 : bq21_3부터 한 칸씩 오른쪽으로

In [51]:
meta2019[meta2019.knowcode == 158501]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,kq6_1,kq6_2,kq7_1,kq7_2,kq8_1,kq8_2,kq9_1,kq9_2,kq10_1,kq10_2,kq11_1,kq11_2,kq12_1,kq12_2,kq13_1,kq13_2,kq14_1,kq14_2,kq15_1,kq15_2,kq16_1,kq16_2,kq17_1,kq17_2,kq18_1,kq18_2,kq19_1,kq19_2,kq20_1,kq20_2,kq21_1,kq21_2,kq22_1,kq22_2,kq23_1,kq23_2,kq24_1,kq24_2,kq25_1,kq25_2,kq26_1,kq26_2,kq27_1,kq27_2,kq28_1,kq28_2,kq29_1,kq29_2,kq30_1,kq30_2,kq31_1,kq31_2,kq32_1,kq32_2,kq33_1,kq33_2,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13_1,bq13_2,bq13_3,bq14_1,bq14_2,bq14_3,bq14_4,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq22,bq23,bq24,bq25,bq26,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
408,18977,5,5,5,5,5,5,5,5,4,4,4,5,4,4,4,5,4,4,3,5,4,4,4,5,4,4,3,5,3,4,3,4,3,4,4,6,3,3,3,5,3,4,2,4,2,3,2,3,1,0,3,5,2,3,2,4,1,0,3,5,4,5,3,4,3,3,1,0,1,0,2,4,4,4,2,4,2,4,3,5,2,3,13,1,4,1,비파괴검사기사,,,1,4,비파괴 검사에 대한 전반적 지식과 기술교육,6,4,4,4,4,4,4,4,4,4,4,4,4,2.0,4,4.0,5.0,5.0,5.0,4,5.0,2.0,3.0,2.0,2.0,2.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,검사장비의 첨단화,2.0,2,비파괴 종사자 신규유입 부족,2.0,2.0,2,,조선 플랜트 검사회사,,1,49.0,4,재료공학과,1.0,1,,1,40,5000,2200,,158501
466,19035,4,4,4,4,4,4,4,3,3,4,5,5,5,3,3,5,5,4,4,5,3,5,5,5,4,5,4,4,4,3,2,2,3,5,4,4,2,5,4,3,3,3,2,4,3,3,4,3,2,2,2,4,3,4,4,5,2,3,3,3,4,5,3,4,3,3,1,0,2,2,1,0,4,4,5,4,3,4,2,5,3,3,13,1,4,1,초음파비파괴검사기사,침투비파괴검사기사,자기비파괴검사기능사,1,4,"이론, 실무",5,2,4,1,1,1,2,3,1,2,3,3,2,5.0,1,2.0,5.0,5.0,5.0,5,5.0,3.0,4.0,3.0,2.0,2.0,2.0,3.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,"힘든업무(육체적), 위험요소 많음",1.0,3,취업연령이 선호하는 직업이 아님 낮은 급여 위험한 환경,1.0,1.0,1,금융업(대출업무),,,1,35.0,3,사회체육학과,1.0,1,,1,40,3200,2400,,158501
1719,20289,3,4,3,1,4,3,4,5,4,4,3,4,3,3,4,3,1,0,3,5,1,0,1,0,2,3,1,0,1,0,1,0,4,5,2,4,2,3,3,4,4,5,3,4,2,4,3,4,3,4,1,0,2,3,1,0,1,0,1,0,1,0,3,4,3,4,1,0,1,0,1,0,5,6,2,3,1,0,3,4,2,3,13,1,2,1,방사선비파괴검사기능사,,,2,,,6,4,5,4,4,4,4,3,3,3,4,3,3,4.0,3,3.0,3.0,3.0,2.0,3,2.0,3.0,3.0,2.0,2.0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,2.0,3,시설물 기계등의 품질향상을 위한 것이므로 변화가 없어서,3.0,3.0,3,,,,1,30.0,4,방사선학,1.0,1,,1,42,2900,2700,,158501
1783,20353,3,3,3,3,4,4,4,4,4,3,3,3,4,4,4,4,1,0,4,4,1,0,1,0,3,4,1,0,1,0,1,0,3,4,4,4,1,0,1,0,3,4,4,4,3,4,3,4,1,0,1,0,3,4,2,2,1,0,2,3,1,0,2,4,4,4,1,0,1,0,1,0,4,4,3,3,3,4,2,4,1,0,13,1,3,1,비파괴검사기사,,,1,3,장비 사용과 검사 기술에 대한 기술적 교육,3,2,3,3,3,4,4,4,2,2,3,3,3,3.0,3,3.0,5.0,5.0,5.0,3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,,3.0,3,산업 전반에 걸친 안전 진단에 대한 중요성을 늘 강조되고 있어서,3.0,3.0,3,,,,1,30.0,3,정보통신학과,1.0,1,,1,40,2500,2200,,158501
3118,21688,4,4,5,5,5,4,5,5,4,4,5,5,5,5,5,4,3,4,4,2,3,5,3,4,5,4,1,0,1,0,1,0,4,4,3,4,1,0,3,1,4,4,2,4,4,5,1,0,2,2,3,3,2,2,1,0,1,0,2,3,4,3,3,4,4,3,1,0,1,0,1,0,4,5,3,4,3,3,3,4,1,0,13,1,2,1,방사선동위원소취급자 일반면허,초음파투과검사,,1,4,초음파및 방사선 관련교육,4,4,4,4,3,5,5,4,3,4,4,4,4,3.0,4,4.0,3.0,3.0,4.0,4,4.0,3.0,3.0,2.0,2.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,새로운 검사방법의 추가되어 공부를 해야했음,2.0,2,조선업의 경기악화로,3.0,4.0,3,금융계,,,2,49.0,3,방사선학,1.0,1,,1,40,4500,2700,,158501
3231,21801,4,4,5,5,4,4,4,5,4,5,4,5,4,4,4,5,4,3,4,4,4,3,4,4,4,4,4,4,1,0,1,0,4,5,4,5,4,3,4,3,4,4,4,4,4,4,4,4,4,3,4,3,4,3,1,0,1,0,4,4,4,5,4,4,4,2,1,0,4,3,4,4,4,3,4,5,5,6,5,5,1,0,13,1,2,1,방사선 비파괴검사 기사,,,1,2,장비를 다루는 방법들,1,4,4,4,4,1,4,3,3,3,3,3,3,3.0,3,3.0,2.0,3.0,2.0,2,2.0,4.0,4.0,3.0,3.0,3.0,3.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,4.0,1,조선의 경영악학로,4.0,4.0,4,,,,1,47.0,3,방사선학,1.0,1,,1,40,6000,3000,,158501
3361,21931,4,3,3,3,3,3,3,4,3,3,4,4,4,3,4,4,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,4,5,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,3,4,1,0,1,0,1,0,3,4,1,0,1,0,1,0,3,4,1,0,1,0,1,0,1,0,13,1,4,1,비파괴검사국가기술자격,방사성동위원소취급일반면허,"외국비파괴검사자격증(ASNT,EN9712)",1,4,"검사관련 실무, 새로운 검사장비 교육, 국가 기반 사업소(발전소, 플랜트) 공정에 ...",4,4,1,2,1,3,4,2,1,2,4,4,2,4.0,3,2.0,2.0,2.0,2.0,2,2.0,4.0,3.0,3.0,4.0,3.0,3.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,외국NDE업체의 경우 연구

Knowcode : 134303

Index 3361 : bq20부터 한 칸씩 오른쪽으로

In [52]:
meta2019[meta2019.knowcode == 134303]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,kq6_1,kq6_2,kq7_1,kq7_2,kq8_1,kq8_2,kq9_1,kq9_2,kq10_1,kq10_2,kq11_1,kq11_2,kq12_1,kq12_2,kq13_1,kq13_2,kq14_1,kq14_2,kq15_1,kq15_2,kq16_1,kq16_2,kq17_1,kq17_2,kq18_1,kq18_2,kq19_1,kq19_2,kq20_1,kq20_2,kq21_1,kq21_2,kq22_1,kq22_2,kq23_1,kq23_2,kq24_1,kq24_2,kq25_1,kq25_2,kq26_1,kq26_2,kq27_1,kq27_2,kq28_1,kq28_2,kq29_1,kq29_2,kq30_1,kq30_2,kq31_1,kq31_2,kq32_1,kq32_2,kq33_1,kq33_2,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13_1,bq13_2,bq13_3,bq14_1,bq14_2,bq14_3,bq14_4,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq22,bq23,bq24,bq25,bq26,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
1883,20453,5,5,5,5,5,5,5,5,3,3,3,4,5,3,3,3,3,4,4,6,4,5,4,5,4,5,2,3,4,5,4,5,4,6,4,5,4,5,4,5,3,4,3,4,3,4,3,4,3,4,3,5,4,5,4,5,1,0,3,5,3,4,4,5,3,4,3,4,3,4,3,4,3,4,3,5,4,5,3,4,4,5,10,1,2,2,,,,1,2,,3,4,2,2,2,3,3,3,3,3,3,3,3,3.0,3,3.0,4.0,4.0,4.0,4,4.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,,1.0,5,,2.0,2.0,2,,,,1,33.0,3,컴퓨터공학,1.0,1,,1,52,3800,2000,,134303
2430,21000,2,2,2,1,1,1,2,2,2,3,2,2,1,1,1,2,1,0,3,4,1,0,1,0,2,3,1,0,1,0,1,0,4,4,2,3,1,0,1,0,3,4,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,2,2,1,0,1,0,1,0,1,0,1,0,2,3,1,0,1,0,1,0,1,0,13,1,2,1,정보처리기사,리눅스 2급,,1,2,,2,4,4,4,3,3,3,4,4,4,3,4,4,3.0,4,4.0,5.0,4.0,4.0,3,4.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,2.0,4,"기업, 정부 공공기관 업무 생산성 관련 증가 추세",3.0,4.0,2,웹 운영자,,,1,32.0,4,전자공학과,1.0,1,,1,40,2800,2400,,134303
3050,21620,2,2,2,1,2,1,2,2,3,2,2,2,2,1,1,2,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,4,5,3,5,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,3,3,1,0,1,0,1,0,3,4,1,0,1,0,1,0,1,0,1,0,3,4,1,0,1,0,1,0,1,0,13,1,2,1,정보처리기사,,,1,3,,2,5,3,3,3,4,5,3,1,2,3,3,3,3.0,3,2.0,4.0,3.0,3.0,4,4.0,3.0,4.0,3.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,,2.0,4,"커뮤ㅜ터 시스템 감리사라는 전문가 많이 필요. 관리, 감독 기능도 필요하므로",2.0,3.0,3,컴퓨터 감리사,,,1,36.0,4,컴퓨터공학,1.0,1,,1,40,3200,2400,,134303
4288,22859,2,3,4,4,3,2,4,4,4,4,4,4,2,4,4,4,3,4,3,2,3,3,4,3,4,6,2,2,1,0,1,0,1,0,1,0,3,2,1,0,1,0,2,3,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,2,2,2,2,1,0,1,0,1,0,1,0,2,4,2,2,1,0,1,0,16,1,4,2,,,,1,2,"자사 프로그램 교육 및 엑셀 교육, html 등",7,4,4,4,1,1,3,2,2,1,3,3,2,5.0,1,2.0,2.0,2.0,1.0,1,1.0,4.0,5.0,4.0,3.0,1.0,1.0,5.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,프로그램 선을 통한 업무효율을 증대를 통하여 기존의 방식보다 다른부분의 업무영역까지...,3.0,1,사람들의 인식의 변화와 인구수 감소,1.0,1.0,1,시스템 엔지니어(SE),MD나 서비스 운영관리,MD,1,28.0,3,컴퓨터전자,1.0,3,,1,50,3000,2800,,134303
4437,23008,4,3,4,3,4,2,3,3,3,4,4,4,4,3,2,3,2,3,3,4,2,3,2,3,4,5,1,0,1,0,1,0,3,6,1,0,1,0,1,0,2,2,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,2,3,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,10,1,1,2,,,,1,2,"네트워크 기본 , Windows Linux Server 일반",5,4,4,4,4,4,5,3,3,4,4,4,4,2.0,4,4.0,3.0,3.0,3.0,3,3.0,3.0,3.0,2.0,1.0,2.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,효율적인 제공 서비스향샹을 위하여 새로운 구성의 신규 시스템 도입,2.0,2,대기업에서 기존 중소기업의 일자리를 뺏어감,1.0,3.0,2,,IT 컨설팅 분야,,1,48.0,3,정보처리학,2.0,,5,,44,,,5520,134303
5398,23969,2,2,2,1,1,1,2,2,2,2,2,2,1,1,1,2,1,0,3,4,1,0,1,0,3,4,1,0,1,0,1,0,3,4,2,5,1,0,1,0,3,4,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,2,3,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,13,1,2,1,정보처리기사,파워포인트자격증,,1,2,,2,5,3,3,4,3,3,4,2,3,3,4,3,3.0,4,4.0,5.0,5.0,4.0,3,4.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,2.0,4,신규 컴퓨터 시스템 증가 추세로 수요 발생 증가,3.0,3.0,2,,,,1,29.0,5,컴퓨터공학과,1.0,1,,1,40,2700,2400,,134303
5870,24441,3,2,2,1,2,1,2,1,2,3,2,3,1,1,1,2,1,0,2,3,1,0,2,3,2,3,1,0,1,0,1,0,4,5,3,3,1,0,1,0,3,4,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,2,3,1,0,2,3,1,0,1,0,1,0,2,3,1,0,1,0,3,3,1,0,13,1,2,1,정보처리기사,,,1,3,,2,4,4,3,3,3,4,3,2,3,3,4,3,3.0,3,2.0,3.0,4.0,4.0,3,4.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,2.0,4,일정 규모의 시스템 구축시 모두 필요하므로,3.0,3.0,2,웹 엔지니어,,,1,36.0,4,기게공학과,1.0,1,,1,40,3400,2400,,134303
6091,24662,2,2,

Knowcode : 415503

Index 3361 : bq24부터 한 칸씩 뒤로

In [53]:
meta2019[meta2019.knowcode == 415503]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,kq6_1,kq6_2,kq7_1,kq7_2,kq8_1,kq8_2,kq9_1,kq9_2,kq10_1,kq10_2,kq11_1,kq11_2,kq12_1,kq12_2,kq13_1,kq13_2,kq14_1,kq14_2,kq15_1,kq15_2,kq16_1,kq16_2,kq17_1,kq17_2,kq18_1,kq18_2,kq19_1,kq19_2,kq20_1,kq20_2,kq21_1,kq21_2,kq22_1,kq22_2,kq23_1,kq23_2,kq24_1,kq24_2,kq25_1,kq25_2,kq26_1,kq26_2,kq27_1,kq27_2,kq28_1,kq28_2,kq29_1,kq29_2,kq30_1,kq30_2,kq31_1,kq31_2,kq32_1,kq32_2,kq33_1,kq33_2,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13_1,bq13_2,bq13_3,bq14_1,bq14_2,bq14_3,bq14_4,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq22,bq23,bq24,bq25,bq26,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
452,19021,3,3,3,4,4,4,3,3,4,4,3,2,2,3,3,3,1,0,1,0,1,0,1,0,3,4,1,0,1,0,1,0,3,4,3,4,3,4,1,0,3,5,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,2,5,3,6,3,5,1,0,1,0,3,2,1,0,2,2,3,2,3,4,3,3,1,0,10,1,1,2,,,,1,5,"편집교육, 항공촬영 교육",6,2,3,3,3,4,3,3,3,9,9,5,3,3.0,2,3.0,1.0,2.0,1.0,1,1.0,2.0,3.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,새로운 장비도입으로 기술의 급변화,1.0,1,전문성이 떨어짐 평준화,1.0,2.0,1,,,,1,45.0,2,인문계,1.0,1,,1,45,1500,1200,,415503
886,19455,5,4,5,5,4,4,3,4,3,3,4,4,4,2,4,3,3,4,3,3,2,4,2,3,2,3,4,5,1,0,1,0,4,5,2,3,5,6,2,2,2,2,4,5,3,3,2,3,2,2,2,3,2,3,3,4,2,2,2,3,4,6,4,3,3,4,3,4,3,4,2,3,3,4,3,4,3,4,3,4,2,2,18,1,6,2,,,,1,5,그래픽 프로그램사용법,5,4,4,4,3,3,3,1,4,3,3,3,4,3.0,4,4.0,3.0,2.0,3.0,1,2.0,4.0,5.0,5.0,4.0,2.0,2.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,"다양한 디자인 요구와 CG양이 늘어 여러기법 도입, 연구로 업무의 난이도가 높아졌다",2.0,3,기계가 해주는게 늘기도 하지만 디자이너의 아이디어와 영역이 확대되고 있다,1.0,1.0,1,,,,2,42.0,4,산업디자인학과,1.0,1,,1,45,10000,3000,,415503
1637,20207,5,5,5,4,4,3,3,4,5,5,5,5,3,4,5,3,5,4,3,5,4,4,5,4,4,3,5,5,4,4,1,0,3,5,1,0,5,4,1,0,2,2,2,4,2,3,1,0,2,2,4,3,3,2,2,3,1,0,1,0,2,3,4,3,3,3,4,4,2,3,2,3,2,2,2,3,3,4,5,5,2,3,10,4,1,2,,,,1,2,"촬영장비 및 프로그램 사용법 숙지, 회사의 기록 작성법 숙지",6,2,3,2,1,4,5,3,3,9,9,9,5,2.0,4,4.0,4.0,4.0,4.0,3,3.0,3.0,5.0,3.0,3.0,2.0,2.0,4.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,"새로운 컴퓨터 프로그램의 도입 및 사용으로 업무가 좀 더 수월해졌고, 새로운 기계나...",1.0,3,"기계나 장비도 대체될수 없고, 점점 영상미디어의 수요가 늘어날 것으로 전망",1.0,1.0,2,,디자인계열,,2,33.0,3,디지털 영상 디자인과,2.0,,5,,20,,,3400,415503
1656,20226,3,3,4,3,4,3,3,3,3,4,5,3,3,2,3,2,2,2,3,2,2,3,2,2,2,2,2,2,1,0,1,0,2,1,1,0,2,2,1,0,2,2,2,2,2,3,1,0,2,2,2,3,2,3,3,3,2,2,2,3,2,3,2,4,2,3,2,3,2,2,2,2,2,2,2,2,2,4,3,4,2,2,10,1,1,2,,,,1,4,촬영장비 및 편집프로그램 사용법 숙지,1,3,3,3,3,4,4,3,3,4,4,4,3,3.0,3,2.0,3.0,2.0,2.0,2,2.0,3.0,4.0,3.0,3.0,2.0,2.0,4.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,새로운 컴퓨터 프로그램 도입 및 사용으로 업무가 수월해졌다,2.0,3,기계나 장비로 대체할수 없는 부분이 비슷해서,2.0,2.0,2,,,,2,30.0,4,중문학과,1.0,1,,1,48,2240,1800,,415503
3022,21592,4,4,4,3,4,3,4,4,4,4,4,5,3,4,5,3,2,3,2,3,2,2,2,3,3,3,2,2,2,3,1,0,2,4,2,3,3,3,1,0,2,2,2,3,2,2,1,0,2,2,2,3,2,2,2,3,2,2,1,0,2,3,3,3,2,3,2,3,2,2,2,2,2,2,2,2,2,2,3,4,2,3,10,1,1,2,,,,1,2,장비 및 프로그램 교육,7,3,3,1,1,4,4,3,3,4,4,3,4,2.0,4,3.0,3.0,4.0,3.0,3,3.0,3.0,2.0,2.0,2.0,3.0,2.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,더 빠르고 처리가 빠른 장비와 프로그램으로 업무의 흐름이 수월해졌다,1.0,3,영상미디어 수요가 변화 없어서,1.0,2.0,1,미술치료사,,,2,39.0,5,심리치료학과,1.0,1,,1,40,3050,2300,,415503
3309,21879,5,4,5,5,5,4,5,5,4,5,5,5,4,3,3,5,2,1,3,3,1,0,1,0,1,0,1,0,1,0,1,0,3,4,1,0,3,4,1,0,2,2,1,0,1,0,1,0,1,0,2,2,3,4,3,4,1,0,1,0,2,2,5,4,2,3,3,3,3,4,2,2,2,3,2,3,3,5,4,5,2,2,15,2,5,2,,,,1,3,그래픽을 의뢰받고 실제로 구현하여 방송으로 송출되기까지의 프로세스에 익숙해지고 돌방...,5,5,5,3,3,4,5,3,3,3,5,5,4,4.0,4,4.0,4.0,5.0,4.0,2,4.0,4.0,3.0,3.0,4.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,업무시 필요한 그래픽 프로그램의 업그레이드로 인한 기술의 변화가 있었고 그것에 적응...,1.0,3,인공지능이 영상그래픽 작업을 대체할 수 있을 것 같지만 특히나 보도 그래픽의 경우에...,3.0,4.0,,"영상편집감독,인쇄디자인,프리젠테이션디자인",,2,34,4.0,영상영화디자인과,1,1.0,,1,50,3100,2000,,NaN,415503
3516,22086,5,5,5,2,3,3,2,4,5,5,4,4,2,5,2,3,1,0,4,3,1,0,1,0,1,0,1,0,1,0,1,0,3,4,2,2,3,3,1,0,2,2,2,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,4,4,3,2,2,2,2,2,2,2,1,0,1,0,3,4,4,4,1,0,10,2,5,2,,,,1,3

Knowcode : 413202

Index 3361 : bq6에서 bq24로 이동


In [54]:
meta2019[meta2019.knowcode == 413202]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,kq6_1,kq6_2,kq7_1,kq7_2,kq8_1,kq8_2,kq9_1,kq9_2,kq10_1,kq10_2,kq11_1,kq11_2,kq12_1,kq12_2,kq13_1,kq13_2,kq14_1,kq14_2,kq15_1,kq15_2,kq16_1,kq16_2,kq17_1,kq17_2,kq18_1,kq18_2,kq19_1,kq19_2,kq20_1,kq20_2,kq21_1,kq21_2,kq22_1,kq22_2,kq23_1,kq23_2,kq24_1,kq24_2,kq25_1,kq25_2,kq26_1,kq26_2,kq27_1,kq27_2,kq28_1,kq28_2,kq29_1,kq29_2,kq30_1,kq30_2,kq31_1,kq31_2,kq32_1,kq32_2,kq33_1,kq33_2,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13_1,bq13_2,bq13_3,bq14_1,bq14_2,bq14_3,bq14_4,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq22,bq23,bq24,bq25,bq26,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
534,19103,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,5,4,6,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,3,3,2,3,4,3,5,3,4,3,3,3,4,3,4,3,4,3,2,3,3,3,4,3,3,3,3,3,3,3,4,3,4,3,4,3,4,3,4,3,4,3,4,15,2,6,1,기록물관리전문요원 자격,,,1,2,해외 기록관리 사례,6,6,4,5,5,5,5,5,4,4,4,5,4,1.0,5,5.0,5.0,5.0,5.0,4,4.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,기록관리시스템 개발,2.0,4,민간영역에서의 다양한 기록활동이 이루어지고 있으므로,2.0,3.0,2,,,마을기록활동가,2,34.0,5,기록관리학,1.0,1,,1,40,3500,3000,,413202
1465,20034,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,4,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,2,3,2,3,2,3,2,2,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,15,2,5,1,기록물관리전문요원,,,1,2,,6,6,3,3,4,3,3,3,3,3,3,3,3,3.0,3,3.0,4.0,4.0,4.0,4,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,,2.0,3,,3.0,3.0,3,,,,1,47.0,5,기록관리학,1.0,1,,1,40,8000,,,413202
1693,20263,5,5,5,5,5,5,5,5,4,5,5,5,5,4,5,5,3,4,3,4,3,4,3,4,3,4,3,2,3,4,3,4,3,5,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,3,4,4,5,3,5,3,3,5,5,3,4,5,6,5,5,5,5,5,5,4,4,15,2,2,1,기록관리전문요원자격증,정교사2급자격증,한국사능력평가1급,1,1,민간기록수집업무,2,6,4,4,4,5,5,3,4,4,3,3,4,1.0,5,4.0,4.0,4.0,3.0,2,2.0,3.0,4.0,3.0,3.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,지방 최초로 경상남도기록원(영구기록물관리기관)이 생겼다,2.0,4,기록관리 관련 법 개정으로 기록연구사가 각 기관마다 배치인원이 늘어날것이기때문,2.0,2.0,3,중학교 교사,중학교 역사교사,,2,36.0,5,기록관리학,1.0,1,,1,40,3700,3700,,413202
1824,20394,4,4,5,4,5,4,5,4,3,4,5,5,5,2,2,3,4,5,5,7,1,0,1,0,3,4,3,4,1,0,1,0,4,5,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,2,3,1,0,1,0,1,0,3,4,3,4,2,4,1,0,4,5,3,3,4,4,4,5,1,0,3,4,2,2,15,2,6,1,기록물관리전문요원 자격,,,1,2,1. 기록물 관리 일반 사항_2. 기록물 선별 및 평가 등_3. 전자기록물 관리 관...,5,6,5,4,3,5,4,4,3,3,4,5,4,2.0,4,3.0,5.0,5.0,5.0,4,4.0,1.0,3.0,2.0,1.0,1.0,1.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,2.0,3,법령 상 기록물관리전문요원 채용 의무가 있는 대부분의 기관에서 1명 이상의 채용을 ...,2.0,3.0,2,기록물관리전문요원,,,1,30.0,5,기록학,1.0,1,,1,40,3500,3000,,413202
2610,21180,5,5,5,2,5,2,5,3,3,2,5,5,5,5,2,3,2,4,4,6,1,0,2,2,2,3,2,2,1,0,1,0,2,5,2,3,3,2,2,2,2,2,2,2,2,2,2,3,2,2,2,3,4,5,2,3,1,0,3,5,4,6,4,5,4,4,4,3,5,5,5,6,4,4,5,4,3,4,4,4,2,2,15,2,3,1,기록물관리전문요원 자격증,,,1,4,· 정부조직론_· 대한민국정부사(史)_· 한국기록관리제도사(史)_· 국내 근현대 고...,"데이터마이너""",1,30,5,정보·기록학,1,1,,1,44,4300,2400,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,413202
3378,21948,4,4,4,2,4,3,4,3,3,3,4,4,4,2,2,3,1,0,3,2,1,0,1,0,1,0,1,0,1,0,1,0,2,3,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,3,3,2,2,1,0,4,4,1,0,1,0,3,2,2,3,3,2,1,0,15,2,2,1,사서,,,2,,,5,5,4,4,4,4,4,4,3,3,3,4,3,2.0,4,4.0,4.0,3.0,3.0,3,3.0,3.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,1.0,3,기계나 컴퓨터가 한계가 있어서,2.0,2.0,2,,,,2,38.0,4,문헌정보,1.0,1,,1,40,3000,2500,,413202
3911,22481,4,4,5,1,4,2,3,3,4,3,4,4,4,2,1,2,1,0,2,3,1,0,1,0,1,0,1,0,1,0,1,0,3,4,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,3,3,1,0,1,0,1,0,3,2,4,5,3,3,1,0,4,3,1,0,1,0,1,0,2,3,4,3,1,0,15,2,2,1,기록물관리전문요원 자격,,,2,,,4,5,4,4,4,4,4,4,3,3,3,4,3,2.0,4,4.0,3.0,3.0,2.0,3,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,2.0,2,자동화 시스템,3.0,3.0,3,,,

Knowcode : 132002

Index 614 : bq20부터 한 칸 뒤로

In [55]:
meta2019[meta2019.knowcode == 132002]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,kq6_1,kq6_2,kq7_1,kq7_2,kq8_1,kq8_2,kq9_1,kq9_2,kq10_1,kq10_2,kq11_1,kq11_2,kq12_1,kq12_2,kq13_1,kq13_2,kq14_1,kq14_2,kq15_1,kq15_2,kq16_1,kq16_2,kq17_1,kq17_2,kq18_1,kq18_2,kq19_1,kq19_2,kq20_1,kq20_2,kq21_1,kq21_2,kq22_1,kq22_2,kq23_1,kq23_2,kq24_1,kq24_2,kq25_1,kq25_2,kq26_1,kq26_2,kq27_1,kq27_2,kq28_1,kq28_2,kq29_1,kq29_2,kq30_1,kq30_2,kq31_1,kq31_2,kq32_1,kq32_2,kq33_1,kq33_2,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13_1,bq13_2,bq13_3,bq14_1,bq14_2,bq14_3,bq14_4,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq22,bq23,bq24,bq25,bq26,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
614,19183,5,5,5,3,5,5,3,5,5,3,3,5,3,3,5,5,1,0,1,0,2,2,1,0,3,4,2,4,1,0,1,0,4,6,3,4,3,5,1,0,3,4,3,4,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,3,4,3,4,4,5,1,0,1,0,1,0,2,3,2,3,3,4,1,0,1,0,10,1,2,2,,,,1,5,"프로그래밍 언어, 네트워크 및 데이터베이스 이해_",6,5,2,1,1,2,4,2,2,3,3,3,3,3.0,4,4.0,1.0,2.0,1.0,1,2.0,4.0,4.0,4.0,4.0,3.0,3.0,5.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,예시 1) 계속해서 빠르게 신기술 나와서 스스로 혼자 독학하며 익혀야 했음.__예시...,3.0,기계 또한 프로그램 언어로 돌아가는 거니까.,2,2.0,2.0,,,,2,26,3.0,모바일인터넷,1,1.0,,1,45,2600.0,2200,,NaN,132002
673,19242,5,3,5,3,5,3,5,5,2,5,3,5,5,4,5,5,2,1,2,3,1,0,1,0,3,4,1,0,1,0,1,0,5,6,1,0,1,0,1,0,3,2,2,5,1,0,1,0,1,0,1,0,1,0,1,0,1,0,3,4,3,5,1,0,3,4,1,0,1,0,1,0,4,4,1,0,5,5,4,4,1,0,10,1,2,2,,,,1,4,시스템 구축에 대한 전문적이고 폭넓은 교육,4,3,4,4,3,5,5,5,4,4,9,9,5,3.0,5,3.0,5.0,4.0,5.0,3,6.0,3.0,3.0,4.0,3.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,개발언어 변화로 인한 사용언어 변경(추가).기존 서비스에 새로운 업무를 추가하여 업무증진,2.0,4,자동화로인한 수요증가,3.0,4.0,4,자영업자,"웹퍼블리셔,프론트엔드 개발자,플스텍 개발자 등",,1,33.0,3,전산,1.0,1,,1,40.0,2900,2400,,132002
684,19253,4,3,4,3,4,3,4,4,4,4,5,4,5,3,3,4,1,0,2,3,1,0,1,0,1,0,1,0,1,0,1,0,3,5,3,5,3,3,1,0,1,0,3,4,3,4,1,0,1,0,1,0,1,0,1,0,1,0,1,0,3,4,3,4,3,4,1,0,1,0,1,0,3,4,3,3,4,5,4,4,4,4,10,1,2,1,정보통신기사,,,2,,,3,4,3,3,3,4,3,3,2,2,4,4,3,4.0,1,2.0,3.0,3.0,5.0,3,4.0,3.0,4.0,2.0,2.0,2.0,2.0,4.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,전력량계 제조개발,2.0,3,IT산업의 규모가 증가하고 있어서,3.0,3.0,2,방송장비,,,1,28.0,3,전기과,1.0,1,,1,40.0,2600,2500,,132002
848,19417,4,4,4,5,4,2,2,4,5,4,5,4,5,2,3,5,1,0,3,5,1,0,1,0,3,5,1,0,1,0,1,0,3,5,2,4,1,0,1,0,1,0,2,4,1,0,1,0,1,0,1,0,1,0,1,0,1,0,2,3,3,4,1,0,3,5,1,0,1,0,1,0,1,0,1,0,2,5,2,4,1,0,10,1,3,1,DAP,,,1,5,,6,5,3,2,2,4,4,3,3,3,3,3,3,2.0,3,3.0,3.0,4.0,4.0,5,5.0,3.0,4.0,3.0,3.0,2.0,3.0,5.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,"신규기계나 서비스 도입시 시스템, DB 설계 작업이 필요한 경우가 있음",1.0,3,설계나 분석의 경우 조금씩 방법이 달라 사람이 필요,2.0,2.0,4,,,,1,46.0,4,컴퓨터공학,1.0,1,,1,47.0,,,,132002
1753,20323,3,3,4,5,4,2,4,5,5,4,5,5,5,4,3,5,4,4,5,5,3,5,2,5,4,4,2,5,4,5,2,2,5,6,4,5,4,5,2,4,3,3,4,5,3,4,3,4,3,3,3,4,3,4,4,5,2,3,4,3,4,4,5,5,4,4,2,4,2,4,1,0,5,6,4,5,5,6,4,5,3,4,10,1,4,1,정보시스템감리원증,정보시스템기사,,1,1,요구자격에 대한 보수교육,6,5,4,3,3,5,4,3,2,3,3,3,3,3.0,4,4.0,4.0,3.0,4.0,3,4.0,2.0,3.0,4.0,3.0,2.0,3.0,4.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,AI(인공지능) 및 4차산업 도입이 늘어남에 따른 정보시스템 감리에 대한 제도 또는...,3.0,1,"정보시스템감리 무용론 업계에서 거론되고 있음, 정보시스템 구축 및 외부 환경변화에 ...",5.0,5.0,5,금융건 전산직군 근무,정보시스템 도입(구축)컨설팅,빅데이터 분석가,1,64.0,4,전자계산학과,1.0,4,,2,40.0,2500,2200,,132002
2838,21408,3,4,4,4,5,3,2,3,3,4,4,3,4,2,2,2,3,2,2,5,2,5,3,4,4,6,1,0,3,5,2,3,4,5,2,4,1,0,1,0,2,3,3,5,2,5,1,0,1,0,2,3,1,0,1,0,1,0,2,5,3,5,2,5,2,5,1,0,1,0,1,0,3,5,3,4,3,5,3,5,2,3,10,1,2,2,,,,1,3,,6,5,3,3,3,3,4,3,3,3,3,3,3,3.0,3,4.0,4.0,3.0,2.0,3,5.0,4.0,4.0,5.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,모바일 테스트 자료화 도입,2.0,5,시스템 환경 변화 대응할 설계 분석 필요,3.0,3.0,2,,,,1,43.0,4,정보통신공학,1.0,1,,1,45.0,5800,3000,,132002
3892,22462,3,3,3,4,3,3,3,3,3,3,3,4,3,2,2,3,3,4,2,4,2,4,2,3,3,3,2,3,2,3,1,0,3,4,2,3,1,0,1,0,2,3,1,0,1,0,1,0,1,0,2,3,1,0,1,0,1,0,2,4,2,4,1,0,2,3,1,0,1,0,1,0,1,0,2,3,3,4,3,4,1,0,10,1,2,1,정보처리기사,,,1,2,,6,5,3,3,3,2,3,2,3,3,3,3,3,2.0,3,3.0,2.0,2.0,2.0,1,2.0,3.0,3.0,4.0,3.0

Knowcode : 158501

Index 177 : bq20~bq21-3을 한 칸씩 오른쪽으로
bq24부터 한 칸씩 뒤로

In [56]:
meta2019[meta2019.knowcode == 151201]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,kq6_1,kq6_2,kq7_1,kq7_2,kq8_1,kq8_2,kq9_1,kq9_2,kq10_1,kq10_2,kq11_1,kq11_2,kq12_1,kq12_2,kq13_1,kq13_2,kq14_1,kq14_2,kq15_1,kq15_2,kq16_1,kq16_2,kq17_1,kq17_2,kq18_1,kq18_2,kq19_1,kq19_2,kq20_1,kq20_2,kq21_1,kq21_2,kq22_1,kq22_2,kq23_1,kq23_2,kq24_1,kq24_2,kq25_1,kq25_2,kq26_1,kq26_2,kq27_1,kq27_2,kq28_1,kq28_2,kq29_1,kq29_2,kq30_1,kq30_2,kq31_1,kq31_2,kq32_1,kq32_2,kq33_1,kq33_2,bq1,bq2,bq3,bq4,bq4_1a,bq4_1b,bq4_1c,bq5,bq5_1,bq5_2,bq6,bq7,bq8_1,bq8_2,bq8_3,bq9,bq10,bq11,bq12_1,bq12_2,bq12_3,bq12_4,bq12_5,bq13_1,bq13_2,bq13_3,bq14_1,bq14_2,bq14_3,bq14_4,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq22,bq23,bq24,bq25,bq26,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
153,18722,5,5,5,3,5,5,5,5,5,5,5,5,5,5,5,5,3,4,5,5,2,2,5,6,2,2,1,0,4,3,1,0,5,6,5,7,5,7,1,0,5,7,5,7,4,6,2,4,2,4,1,0,1,0,1,0,1,0,1,0,5,6,5,6,5,6,1,0,1,0,1,0,5,3,1,0,4,5,2,5,3,3,13,2,6,2,,,,2,,,6,6,5,5,5,4,5,3,2,9,5,5,4,3.0,4,3.0,3.0,1.0,5.0,2,4.0,4.0,5.0,2.0,4.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,인공지능 등 기존 로봇 분야에 접목할 새로운 기술의 발전,1.0,3,연구직으로 정부 정책으로 연구소를 줄이지 않으면 변화 없을 것으로 생각됨,2.0,3.0,2,,"기계 설계, 제어 등등",,1,35.0,6,기계공학과,1.0,2,,2,52,4500,3400,,151201
169,18738,4,2,4,3,4,4,4,4,4,4,4,4,4,3,4,4,3,4,4,3,2,3,1,0,4,2,1,0,4,4,1,0,3,4,4,6,3,5,1,0,4,6,4,6,4,7,2,5,1,0,1,0,3,4,1,0,1,0,1,0,1,0,3,5,4,5,2,2,1,0,1,0,4,4,1,0,4,4,4,5,4,2,13,2,4,2,,,,1,2,,6,6,4,4,2,4,5,5,5,5,5,5,5,2.0,4,3.0,5.0,5.0,5.0,3,5.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,1.0,5,로봇 산업이 미래 산업으로 주목받은 지 오래 되어 현재 관련 사업 분야의 성장 속도...,2.0,4.0,2,,,,1,27.0,4,기계공학,1.0,2,,2,50,3400,3100,,151201
177,18746,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,4,4,3,4,3,4,3,3,3,3,3,3,3,3,3,3,3,4,4,5,3,4,3,5,4,5,5,5,5,5,4,4,3,2,3,2,3,3,3,3,2,3,2,3,2,3,3,3,2,3,3,3,2,3,3,3,2,3,2,3,2,4,3,4,2,3,13,2,4,2,,,,1,5,"기술 발전에 따른 AI 심도 학습, 기술 트랜드 교육 등",6,6,4,4,4,2,4,4,3,3,3,3,3,5.0,1,3.0,2.0,2.0,5.0,2,2.0,3.0,5.0,4.0,4.0,3.0,4.0,3.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,4차 산업혁명이라는 키워드로 로봇에 대한 관심과 기대가 크게 증가 되었다._제조 공...,2.0,"어떠한 분야든 자동화가 도입이 되면, 인력 감축을 불가피하며, 대신 자동화 설비 유...",1,2.0,1.0,로봇 엔지니어,,로봇 및 자동화 시설 유지관리 분야,1,43,5.0,지능로봇학,1,1.0,,1,60,6000,4000,,NaN,151201
370,18939,4,3,4,3,4,4,4,3,3,4,4,4,4,3,5,5,3,4,3,5,3,5,3,5,3,4,3,5,3,5,3,4,4,7,4,7,3,6,3,5,4,6,4,6,4,5,3,5,2,2,3,5,3,2,3,6,3,4,3,3,3,6,4,5,3,6,3,4,3,5,2,2,4,7,3,4,4,7,3,5,3,4,13,2,4,2,,,,1,5,"전자회로도 작성, PCB설계, 디버깅, 펌웨어 등",6,5,4,4,4,4,4,3,4,4,4,4,4,3.0,3,3.0,4.0,4.0,4.0,4,5.0,3.0,4.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,"이터켓 필드버스를 사용하여 실시간성 보장한 제어시스템 구축, 3d프린트를 활용한 시...",2.0,3,"로봇이 할수있는 일보다 아직은 할수 없는 일이 많고, 할수 없는일을 할 수 있는 로...",1.0,3.0,1,선박자동회시스템 연구 개발,"전장 구성, 시스템 구성 등. 많은곳에서 활용 가능 할것 같다.",,1,31.0,4,전자공학,1.0,1,,1,50,5000,3000,,151201
524,19093,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,3,4,3,4,3,3,3,4,3,4,3,4,3,4,1,0,4,6,4,6,3,4,3,4,4,5,4,6,4,5,4,5,3,4,3,4,3,4,3,4,2,2,3,4,4,5,4,5,4,5,2,2,2,2,2,2,4,6,3,4,3,5,3,4,3,4,13,2,4,2,,,,2,,,4,5,4,4,4,4,4,4,4,4,4,4,4,4.0,4,4.0,5.0,5.0,5.0,5,5.0,3.0,4.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,인사발령 有,2.0,3,,2.0,2.0,2,,,,2,26.0,4,정보통신,1.0,1,,1,40,4000,3500,,151201
590,19159,3,4,4,4,5,5,5,4,4,4,4,3,5,3,5,4,3,4,4,4,4,5,3,6,4,5,4,5,5,7,3,4,5,7,5,7,5,7,4,6,5,7,5,7,5,7,4,5,4,5,4,6,4,5,3,4,5,6,4,5,3,4,3,4,5,7,3,3,3,3,2,3,5,5,3,4,4,5,3,6,4,6,13,2,4,2,,,,2,,,6,6,3,3,3,4,4,4,3,3,3,3,3,3.0,3,3.0,2.0,1.0,1.0,2,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,2.0,2,,3.0,4.0,4,엔지니어,,,1,39.0,6,기계공학과,1.0,1,,1,48,8000,5000,,151201
2290,20860,3,4,4,2,2,2,2,4,4,4,5,5,3,2,2,5,2,5,2,4,2,4,2,6,2,4,4,5,2,4,1,0,5,7,5,7,2,3,1,0,4,5,4,5,4,4,2,3,1,0,1,0,1,0,1,0,1,0,1,0,3,4,2,4,5,6,1,0,1,0,1,0,5,4,2,4,5,6,2,4,1,0,13,2,6,2,,,,1,2,,4,6,5,4,4,4,5,3,3,3,3,4,4,2.0,4,4.0,4.0,4.0,3.0,3,2.0,3.0,4.0,3.0,3.0,3.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,딥러닝 기

2020년도 진행

In [57]:
c_list = meta2020.columns.tolist()
nan2020 = []

for i in c_list:
    nan2020.append([i,list(meta2020[meta2020[i].isnull()].index)])
nan2020

[['idx', []],
 ['saq1_1', []],
 ['saq1_2', []],
 ['saq2_1', []],
 ['saq2_2', []],
 ['saq3_1', []],
 ['saq3_2', []],
 ['saq4_1', []],
 ['saq4_2', []],
 ['saq5_1', []],
 ['saq5_2', []],
 ['saq6_1', []],
 ['saq6_2', []],
 ['saq7_1', []],
 ['saq7_2', []],
 ['saq8_1', []],
 ['saq8_2', []],
 ['saq9_1', []],
 ['saq9_2', []],
 ['saq10_1', []],
 ['saq10_2', []],
 ['saq11_1', []],
 ['saq11_2', []],
 ['saq12_1', []],
 ['saq12_2', []],
 ['saq13_1', []],
 ['saq13_2', []],
 ['saq14_1', []],
 ['saq14_2', []],
 ['saq15_1', []],
 ['saq15_2', []],
 ['saq16_1', []],
 ['saq16_2', []],
 ['saq17_1', []],
 ['saq17_2', []],
 ['saq18_1', []],
 ['saq18_2', []],
 ['saq19_1', []],
 ['saq19_2', []],
 ['saq20_1', []],
 ['saq20_2', []],
 ['saq21_1', []],
 ['saq21_2', []],
 ['saq22_1', []],
 ['saq22_2', []],
 ['saq23_1', []],
 ['saq23_2', []],
 ['saq24_1', []],
 ['saq24_2', []],
 ['saq25_1', []],
 ['saq25_2', []],
 ['saq26_1', []],
 ['saq26_2', []],
 ['saq27_1', []],
 ['saq27_2', []],
 ['saq28_1', []],
 ['saq28_2', [

중간에 Nan 있는 컬럼이 있으나
원래 결측 들어가야할곳이 결측처리된거라
따로 진행 안하고 감

##1-5. 이상치 처리
- 위에 해당되는 이상치들은 설문지.xlsx에 저장
- 파이썬으로 직접하기엔 번거로운게 많고 수작업으로 하는게 이상적이라 수작업으로 진행